<a name="top"></a>Vocab Analysis 
***
# Section 2: Prepare the Data
## [A. Cards](#cards)
- 2.1.1. Import card data into data frame "df_cards"
- 2.1.2. Confirm that card data model matches expected format
- 2.1.3. Shallow check for duplicates (matching rows)
- 2.1.4. Remove unneeded card dataframe columns, rename 'id' to 'cid' (card id)
- 2.1.5. Generate Card Creation Date from Card ID
- 2.1.6. Remove cards with no study data associated with them, cards that have been suspended from study
- 2.1.7. Confirm that no cards considered 'in learning' are present
- 2.1.8. Generate Due Date from Due Value
- 2.1.9. Label card types by their names, & drop outlier
- 2.1.10. Create interval quartile sections for visualization purposes
- 2.1.11. Create dummy variables for card types
- 2.1.12. Create df_cards_009_final_section_2 data frame for progress saving
- 2.1.13. Export df_cards_009_final_section_2

## [B. Notes](#notes)


## [C. Combo](#combo)


### [bottom of page](#bottom)

### 2.0.1. Import libraries

In [1469]:
import pandas as pd
import sqlite3
import json
from datetime import datetime, timedelta, date
import time

### 2.0.2. Import Data

In [1470]:
location = "datasets/collection.anki2"
cnx = sqlite3.connect(location) # create sql file connection

In [1471]:
# TDD backbone assertion to confirm a function call returns the desired result
def assertEquals(actual, expected, desc):
    assert(actual==expected), desc + " result: " + str(actual) + ", expected: " + str(expected)
    return "OK"

### 2.0.3. Extract Deck Creation Date

In [1472]:
df_c = pd.read_sql_query("SELECT * FROM col", cnx)
crt = df_c['crt'][0] # save collection creation date (in epoch time)
pd_crt = pd.to_datetime(crt, unit = 's')
print(pd_crt)

assertEquals(str(pd_crt), "2013-01-08 09:00:00", "Collection Creation Date")

2013-01-08 09:00:00


'OK'

### 2.0.4. Extract field names to label columns

In [1473]:
field_names = []
for row_index, blob in df_c['models'].items():
    for model_id, data in json.loads(blob).items():
        field_names += list(map(lambda fld: fld['name'], data['flds']))
field_names.append('Tags')
expected_names = ['Term', 'Yomi1', 'Translation', 'Translation2', 'Translation3', 'AlternateForms',
    'PartOfSpeech', 'Sound', 'Sound2', 'Sound3', 'Examples', 'ExamplesAudio', 'AtoQ', 'AtoQaudio',
    'AtoQkana', 'AtoQtranslation', 'QandApicture', 'answerPicture', 'Meaning1', 'SimilarWords',
    'RelatedWords', 'Breakdown1', 'Comparison', 'Usage', 'Prompt1', 'Prompt2', 'KakuMCD', 'IuMCD',
    'ExtraMemo', 'Yomi2', 'Meaning2', 'Breakdown2', 'Picture1', 'Picture2', 'Picture3', 'Picture4',
    'HinshiMarker', 'Hint', 'Term2','ArabicNumeral', 'CounterKanji', 'Mnemonic', 'SameSoundWords',
    'Yomi3', 'gChap', 'gBook', 'semester', 'gNumber', 'Transliteration','SoloLookCards',
    'TagOverflow', 'blank1', 'blank2', 'Tags']

In [1474]:
assertEquals(field_names, expected_names, "Field Names")

'OK'

***
- [Back to the top](#top)
- [Next section: Notes](#notes)
***

# <a name="cards"></a> Cards

### 2.1.1. Import card data into data frame "df_cards"

In [1475]:
# Step 6: Take in study data from Anki collection
df_cards = pd.read_sql_query("SELECT * FROM cards", cnx)
assertEquals(df_cards.shape[0],19275,"Rows")#6386, 21979, 19363, 19314
assertEquals(df_cards.shape[1],18,"Columns")

'OK'

### 2.1.2. Confirm that card data model matches expected format

In [1476]:
expected_columns_1 = ['id', 'nid', 'did', 'ord', 'mod', 'usn', 'type', 'queue', 'due', 'ivl', 'factor',
 'reps', 'lapses', 'left', 'odue', 'odid', 'flags', 'data']

def lists_equal(a,b):
    return (a == b).all()

assertEquals(lists_equal(df_cards.columns.values, expected_columns_1), True, "Card Columns Import")

'OK'

### 2.1.3.  Shallow check for duplicates (matching rows)

In [1477]:
 def has_dupes(df_in):
    dupe = df_in.duplicated()
    return df_in.loc[dupe].shape[0] != 0

In [1478]:
assertEquals(has_dupes(df_cards), False, "Duplicates Not Found")

'OK'

### 2.1.4.  Remove unneeded card dataframe columns, rename 'id' to 'cid' (card id)

In [1479]:
def print_line_break():
    print("-"*75)

In [1480]:
def print_before_after(b, a, t=""):
    if t != "":
        print_line_break()
        print(t)
    print_line_break()
    print("Before: " + str(b))
    print_line_break()
    print("After: " + str(a))
    print_line_break()

In [1481]:
df_cards_001_less_cols = df_cards.copy()
df_cards_001_less_cols = df_cards_001_less_cols.drop(['did','usn','type','mod','left','odue','odid','flags','data'],axis=1)
df_cards_001_less_cols = df_cards_001_less_cols.rename(columns={'id':'cid'})
expected_columns_2 = ['cid', 'nid', 'ord', 'queue', 'due', 'ivl', 'factor', 'reps','lapses']

print_before_after(df_cards.columns.values, df_cards_001_less_cols.columns.values,"Card Columns:")

assertEquals(lists_equal(df_cards_001_less_cols.columns.values, expected_columns_2), True, "Card Model Slimmed")

---------------------------------------------------------------------------
Card Columns:
---------------------------------------------------------------------------
Before: ['id' 'nid' 'did' 'ord' 'mod' 'usn' 'type' 'queue' 'due' 'ivl' 'factor'
 'reps' 'lapses' 'left' 'odue' 'odid' 'flags' 'data']
---------------------------------------------------------------------------
After: ['cid' 'nid' 'ord' 'queue' 'due' 'ivl' 'factor' 'reps' 'lapses']
---------------------------------------------------------------------------


'OK'

### 2.1.5. Generate Card Creation Date from Card ID

In [1482]:
df_cards_002_created_date = df_cards_001_less_cols.copy()
df_cards_002_created_date['CardCreated'] = pd.to_datetime(df_cards_002_created_date['cid'],unit='ms')
#df_cards_002_created_date['CardCreated'] = df_cards_002_created_date['CardCreated'].dt.date

#assertEquals(str(df_cards_002_created_date['CardCreated'].iloc[0]), "2012-03-15", "Card ID is in datetime date format year-month-day")

### 2.1.6. Remove cards with no study data associated with them, cards that have been suspended from study

In [1483]:
#queue           integer not null,
#      -- -3=sched buried, -2=user buried, -1=suspended,
#      -- 0=new, 1=learning, 2=due (as for type)

df_cards_003_no_new = df_cards_002_created_date.copy()
df_cards_003_no_new = df_cards_003_no_new[df_cards_003_no_new['queue']!=0] # remove cards marked as new
df_cards_003_no_new = df_cards_003_no_new[df_cards_003_no_new['reps']!=0] # remove cards that have not been reviewed
df_cards_003_no_new = df_cards_003_no_new[df_cards_003_no_new['queue']!=-1] # remove cards that are currently suspended
# https://stackoverflow.com/questions/18196203/how-to-conditionally-update-dataframe-column-in-pandas
df_cards_003_no_new.loc[df_cards_003_no_new['due'] > 10000, 'due'] = 0 # assign 0 to the due # todo: update w/ last studied date from revlog # todo: comment this line out once you have updated the collection import

print_before_after(df_cards_002_created_date.shape[0], df_cards_003_no_new.shape[0],"Card Rows:")

df_cards_003_no_new.tail(5)

---------------------------------------------------------------------------
Card Rows:
---------------------------------------------------------------------------
Before: 19275
---------------------------------------------------------------------------
After: 8260
---------------------------------------------------------------------------


,cid,nid,ord,queue,due,ivl,factor,reps,lapses,CardCreated
18853,1549415185907,1549184119039,2,2,2388,87,2410,3,0,2019-02-06 01:06:25.907
18854,1550403009251,1550402953788,0,2,2399,91,2410,4,0,2019-02-17 11:30:09.251
18855,1550403009269,1550402953788,2,2,2317,9,2010,9,2,2019-02-17 11:30:09.269
18856,1550403084990,1550403040864,0,2,2383,82,2410,3,0,2019-02-17 11:31:24.990
18857,1550403085003,1550403040864,2,2,2401,93,2410,4,0,2019-02-17 11:31:25.003


In [1484]:
df_cards_003_no_new.dtypes

cid                     int64
nid                     int64
ord                     int64
queue                   int64
due                     int64
ivl                     int64
factor                  int64
reps                    int64
lapses                  int64
CardCreated    datetime64[ns]
dtype: object

### 2.1.7. Confirm that no cards considered 'in learning' are present

In [1485]:
sel3 = df_cards_003_no_new[df_cards_003_no_new['due'] == 0]

assertEquals(sel3.shape[0],0,"There are no cards currently in 'learning'.")

'OK'

### 2.1.8. Generate Due Date from Due Value

In [1486]:
df_cards_004_due_date = df_cards_003_no_new.copy()
df_cards_004_due_date['DueDate'] = pd_crt + df_cards_004_due_date['due'].map(timedelta)
#df_cards_004_due_date['DueDate'] = df_cards_004_due_date['DueDate'].dt.date

#assertEquals(str(df_cards_004_due_date['DueDate'].iloc[0]), "2015-03-08", "Card due date is in datetime date format year-month-day")

df_cards_004_due_date.head()

,cid,nid,ord,queue,due,ivl,factor,reps,lapses,CardCreated,DueDate
0,1331799797110,1331799797110,0,2,789,65,1680,10,1,2012-03-15 08:23:17.110,2015-03-08 09:00:00
1,1331799797112,1331799797112,0,2,784,149,2080,8,1,2012-03-15 08:23:17.112,2015-03-03 09:00:00
3,1331799797114,1331799797114,0,2,757,99,1980,7,0,2012-03-15 08:23:17.114,2015-02-04 09:00:00
5,1331799797117,1331799797117,0,2,798,143,2130,6,1,2012-03-15 08:23:17.117,2015-03-17 09:00:00
6,1331799797118,1331799797118,0,2,759,74,1880,15,3,2012-03-15 08:23:17.118,2015-02-06 09:00:00


In [1487]:
df_cards_004_due_date.dtypes

cid                     int64
nid                     int64
ord                     int64
queue                   int64
due                     int64
ivl                     int64
factor                  int64
reps                    int64
lapses                  int64
CardCreated    datetime64[ns]
DueDate        datetime64[ns]
dtype: object

It appears that card types are being rendered as numbers, which makes it less human readible. We will fix this.   
### 2.1.9. Label card types by their names, & drop outlier

In [1488]:
df_cards_005_named_types = df_cards_004_due_date.copy()
# ord stands for 'ordinal' : identifies which of the card templates it corresponds to
print(df_cards_005_named_types['ord'].value_counts()) # these are the card vectors

# The dataset contains only two cards of a single card vector: let's drop them as outliers
df_cards_005_named_types = df_cards_005_named_types.drop(df_cards_005_named_types[df_cards_005_named_types['ord'] == 11].index)

df_cards_005_named_types['ord'].value_counts() # the check shall pass

# now, to map the names onto the card vectors # read:JapaneseReading, recall:EngToJpnTranslate, look:PictureLook, listen:AudioListening
df_cards_005_named_types['CardType'] = df_cards_005_named_types['ord'].map(
    {0:'read', 2:'recall', 4:'look', 7:'listen'})
df_cards_005_named_types['CardType'].value_counts()

0     6844
4     1121
7      279
2       14
11       2
Name: ord, dtype: int64


read      6844
look      1121
listen     279
recall      14
Name: CardType, dtype: int64

### 2.1.10. Create interval quartile sections for visualization purposes

In [1489]:
df_cards_006_ivl_buckets = df_cards_005_named_types.copy()
# qcut: Quantile-based discretization function. Discretize variable into equal-sized buckets
# based on rank or based on sample quantiles. For example 1000 values for 10 quantiles would
# produce a Categorical object indicating quantile membership for each data point.
# http://www.datasciencemadesimple.com/quantile-decile-rank-column-pandas-python-2/
df_cards_006_ivl_buckets['c_ivl_q'] = pd.qcut(df_cards_006_ivl_buckets['ivl'],5,labels=False)
df_cards_006_ivl_buckets['c_factor_q'] = pd.qcut(df_cards_006_ivl_buckets['factor'],3,labels=False)
df_cards_006_ivl_buckets.head()

,cid,nid,ord,queue,due,ivl,factor,reps,lapses,CardCreated,DueDate,CardType,c_ivl_q,c_factor_q
0,1331799797110,1331799797110,0,2,789,65,1680,10,1,2012-03-15 08:23:17.110,2015-03-08 09:00:00,read,0,1
1,1331799797112,1331799797112,0,2,784,149,2080,8,1,2012-03-15 08:23:17.112,2015-03-03 09:00:00,read,0,2
3,1331799797114,1331799797114,0,2,757,99,1980,7,0,2012-03-15 08:23:17.114,2015-02-04 09:00:00,read,0,1
5,1331799797117,1331799797117,0,2,798,143,2130,6,1,2012-03-15 08:23:17.117,2015-03-17 09:00:00,read,0,2
6,1331799797118,1331799797118,0,2,759,74,1880,15,3,2012-03-15 08:23:17.118,2015-02-06 09:00:00,read,0,1


### 2.1.11. Create dummy variables for card types

In [1490]:
df_cards_007_dummies = df_cards_006_ivl_buckets.copy()

df_cards_007_dummies = pd.get_dummies(df_cards_007_dummies, columns=['CardType'])
df_cards_007_dummies['cardtype'] = df_cards_006_ivl_buckets['CardType']

In [1491]:
df_cards_007_dummies.tail(10)[:5]

,cid,nid,ord,queue,due,ivl,factor,reps,lapses,CardCreated,DueDate,c_ivl_q,c_factor_q,CardType_listen,CardType_look,CardType_read,CardType_recall,cardtype
18811,1523893083493,1523892839900,0,2,2728,420,2210,7,1,2018-04-16 15:38:03.493,2020-06-28 09:00:00,3,2,0,0,1,0,read
18812,1523893083509,1523892839900,7,2,2823,515,2410,5,0,2018-04-16 15:38:03.509,2020-10-01 09:00:00,3,2,1,0,0,0,listen
18813,1523893129423,1523892839900,2,2,2713,405,2210,7,1,2018-04-16 15:38:49.423,2020-06-13 09:00:00,3,2,0,0,0,1,recall
18814,1524841320859,1523892839900,4,2,2308,4,2050,14,2,2018-04-27 15:02:00.859,2019-05-05 09:00:00,0,1,0,1,0,0,look
18852,1549184129288,1549184119039,0,2,2416,108,2410,5,0,2019-02-03 08:55:29.288,2019-08-21 09:00:00,0,2,0,0,1,0,read


### 2.1.12. Create df_cards_008_final_section_2 data frame for progress saving

In [1492]:
df_cards_008_mid_section_2 = df_cards_007_dummies.copy()
# we will also drop a few columns that aren't needed anymore
df_cards_008_mid_section_2 = df_cards_008_mid_section_2.drop(['ord','queue','due'],axis=1)
print(df_cards_008_mid_section_2.shape)
df_cards_008_mid_section_2.head()

(8258, 15)


,cid,nid,ivl,factor,reps,lapses,CardCreated,DueDate,c_ivl_q,c_factor_q,CardType_listen,CardType_look,CardType_read,CardType_recall,cardtype
0,1331799797110,1331799797110,65,1680,10,1,2012-03-15 08:23:17.110,2015-03-08 09:00:00,0,1,0,0,1,0,read
1,1331799797112,1331799797112,149,2080,8,1,2012-03-15 08:23:17.112,2015-03-03 09:00:00,0,2,0,0,1,0,read
3,1331799797114,1331799797114,99,1980,7,0,2012-03-15 08:23:17.114,2015-02-04 09:00:00,0,1,0,0,1,0,read
5,1331799797117,1331799797117,143,2130,6,1,2012-03-15 08:23:17.117,2015-03-17 09:00:00,0,2,0,0,1,0,read
6,1331799797118,1331799797118,74,1880,15,3,2012-03-15 08:23:17.118,2015-02-06 09:00:00,0,1,0,0,1,0,read


In [1493]:
# Calculate card efficiency (lapses/reps) & durability (interval/reps)

In [1494]:
df_cards_009_mid_section_2 = df_cards_008_mid_section_2.copy() ####
df_cards_009_mid_section_2['efficiency'] = df_cards_009_mid_section_2['lapses'] / df_cards_009_mid_section_2['reps']
df_cards_009_mid_section_2['durability'] = df_cards_009_mid_section_2['ivl'] / df_cards_009_mid_section_2['reps']

In [1495]:
df_cards_009_mid_section_2.head()

,cid,nid,ivl,factor,reps,lapses,CardCreated,DueDate,c_ivl_q,c_factor_q,CardType_listen,CardType_look,CardType_read,CardType_recall,cardtype,efficiency,durability
0,1331799797110,1331799797110,65,1680,10,1,2012-03-15 08:23:17.110,2015-03-08 09:00:00,0,1,0,0,1,0,read,0.100000,6.500000
1,1331799797112,1331799797112,149,2080,8,1,2012-03-15 08:23:17.112,2015-03-03 09:00:00,0,2,0,0,1,0,read,0.125000,18.625000
3,1331799797114,1331799797114,99,1980,7,0,2012-03-15 08:23:17.114,2015-02-04 09:00:00,0,1,0,0,1,0,read,0.000000,14.142857
5,1331799797117,1331799797117,143,2130,6,1,2012-03-15 08:23:17.117,2015-03-17 09:00:00,0,2,0,0,1,0,read,0.166667,23.833333
6,1331799797118,1331799797118,74,1880,15,3,2012-03-15 08:23:17.118,2015-02-06 09:00:00,0,1,0,0,1,0,read,0.200000,4.933333


### 2.1.13. Export df_cards_009_mid_section_2

In [1496]:
df_cards_009_mid_section_2.to_csv('datasets/df_cards_009_mid_section_2.csv')

In [1497]:
# todo: assert that no rare, phrase, sentence or question cards remain in the collection of cards (as down below with notes)

***
- [Previous section: Cards](#cards)
- [Next section: Combo](#combo)
***

# <a name="notes"></a> Notes

### 2.2.1. Import notes (terms/words) into data frame "df_notes"

In [1498]:
# let's take in the 'notes' table, and explicitly save the note id ("nid") 
df_notes = pd.read_sql_query("SELECT * FROM notes", cnx)
df_notes = df_notes.rename(columns={'id':'nid'})

In [1499]:
assertEquals(df_notes.shape[0],8375,"Rows") # 2791, 9784, 8403
assertEquals(df_notes.shape[1],11,"Columns")

'OK'

### 2.2.2. Remove (drop) unneeded fields (columns)

In [1500]:
df_notes_old_col_vals = df_notes.columns.values
df_notes = df_notes.drop(['guid','mid','usn','sfld','csum','flags','data'],axis=1)
#print(df_notes.columns.values)
print_before_after(df_notes_old_col_vals, df_notes.columns.values)

---------------------------------------------------------------------------
Before: ['nid' 'guid' 'mid' 'mod' 'usn' 'tags' 'flds' 'sfld' 'csum' 'flags' 'data']
---------------------------------------------------------------------------
After: ['nid' 'mod' 'tags' 'flds']
---------------------------------------------------------------------------


### 2.2.3. Split "fields" column into multiple, assign field names, drop combined col

In [1501]:
def time_it(func, *args, **kwargs):
    start = time.time()
    func(*args, **kwargs)
    end = time.time()
    # https://stackoverflow.com/questions/8885663/how-to-format-a-floating-number-to-fixed-width-in-python
    print("{:.0f}".format((end - start)*1000) + " miliseconds")

In [1502]:
for i in range(0,len(expected_names)-1):
    df_notes[expected_names[i]] = df_notes.flds.str.split('').str.get(i)
assertEquals('flds' in df_notes.columns.values, True, "'flds' Column Found")
df_notes = df_notes.drop(['flds'],axis=1)
assertEquals('flds' not in df_notes.columns.values, True, "'flds' Column Not Found")
print(df_notes.columns.values)

['nid' 'mod' 'tags' 'Term' 'Yomi1' 'Translation' 'Translation2'
 'Translation3' 'AlternateForms' 'PartOfSpeech' 'Sound' 'Sound2' 'Sound3'
 'Examples' 'ExamplesAudio' 'AtoQ' 'AtoQaudio' 'AtoQkana'
 'AtoQtranslation' 'QandApicture' 'answerPicture' 'Meaning1'
 'SimilarWords' 'RelatedWords' 'Breakdown1' 'Comparison' 'Usage' 'Prompt1'
 'Prompt2' 'KakuMCD' 'IuMCD' 'ExtraMemo' 'Yomi2' 'Meaning2' 'Breakdown2'
 'Picture1' 'Picture2' 'Picture3' 'Picture4' 'HinshiMarker' 'Hint' 'Term2'
 'ArabicNumeral' 'CounterKanji' 'Mnemonic' 'SameSoundWords' 'Yomi3'
 'gChap' 'gBook' 'semester' 'gNumber' 'Transliteration' 'SoloLookCards'
 'TagOverflow' 'blank1' 'blank2']


### 2.2.4. Confirm all HTML tags have been removed from note terms & readings

In [1503]:
assertEquals(df_notes[df_notes['Term'].str.contains("</div>")].shape[0],0,"HTML tags have been removed")
assertEquals(df_notes[df_notes['Term'].str.contains("<div>")].shape[0],0,"HTML tags have been removed")
assertEquals(df_notes[df_notes['Term'].str.contains("anki")].shape[0],0,"HTML tags have been removed")
assertEquals(df_notes[df_notes['Yomi1'].str.contains("</span>")].shape[0],0,"HTML tags have been removed")
assertEquals(df_notes[df_notes['Yomi1'].str.contains("</div>")].shape[0],0,"HTML tags have been removed")
assertEquals(df_notes[df_notes['Yomi1'].str.contains("anki")].shape[0],0,"HTML tags have been removed")

'OK'

In [1504]:
# todo: create function for this
# inspect notes that have spaces in the reading field
# df_notes[df_notes['Term'].str.contains(" ")]

### 2.2.5. Check notes for duplicates (shallow check)

In [1505]:
assertEquals(has_dupes(df_notes), False, "Duplicates Not Found")

'OK'

### 2.2.6. Check for duplicates by term field in notes data frame

In [1506]:
def has_dupe_terms(df_in):
    location = df_in['Term'].duplicated()
    return df_in.loc[location].shape[0] != 0

In [1507]:
assertEquals(has_dupe_terms(df_notes), False, "Duplicates Found")

'OK'

### 2.2.7. Confirm that duplicates dataframe is empty (no dups exist)

In [1508]:
dupe = df_notes['Term'].duplicated() #creates list of True/False values
print(df_notes[dupe].shape)
assertEquals(df_notes[dupe].shape[0], 0, "Duplicates dataframe is empty.")

(0, 56)


'OK'

### 2.2.8. Inspect an individual note by its term

In [1509]:
def get_rows_by_value_in_col(df_in, value, col):
    return df_in.loc[df_in[col]==value]

In [1510]:
# Postal service
sel1 = get_rows_by_value_in_col(df_notes, '発明','Term')
sel1

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,Sound,Sound2,Sound3,Examples,ExamplesAudio,AtoQ,AtoQaudio,AtoQkana,AtoQtranslation,QandApicture,answerPicture,Meaning1,SimilarWords,RelatedWords,Breakdown1,Comparison,Usage,Prompt1,Prompt2,KakuMCD,IuMCD,ExtraMemo,Yomi2,Meaning2,Breakdown2,Picture1,Picture2,Picture3,Picture4,HinshiMarker,Hint,Term2,ArabicNumeral,CounterKanji,Mnemonic,SameSoundWords,Yomi3,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2
2696,1354094556789,1557777647,MCD N3 Noun commonWord complete editThis forJ...,発明,はつめい,"<span style=""""><div>invention</div></span>",,,,"Common word, Noun, Suru verb",[sound:pronunciation_ja_発明.mp3][sound:pronunci...,,,飛行機を発明した人は誰ですか？<div>ひこうき を はつめい した ひと は だれ です ...,,,,,,,,<div>［名］(スル)</div><div>１ 今までなかったものを新たに考え出すこと。特...,,,,,"<div><div><!--StartFragment--><div class=""prog...",<div><br /></div>,<div><br /></div>,<div><div>発(めい)する</div><div>彼には発(めい)の才がある</div...,<div>（？）する</div><div>彼には（？）の才がある</div><div>必要は...,,,,,"<img src=""flywheelbicycle-11.jpg"" />","<img src=""chronicle-of-flight1.jpg"" />","<img src=""x_5f4187ab.jpg"" />","<img src=""mercedes1_b2.jpg"" />",,,,,,,,,,,,,,,,,


### Save Point, Commit, Bonfire (for you Souls fans)*

At the point in time of the data extraction where the (meta) tag information is made available, we can treat it to both clarify (rename poorly worded tags) & reduce (delete unneeded tags). Since we now have all fields split into their own columns as well, we can treat (modifiy & improve) the columns as well, in a 1-2 process: (1) Fix the tags & (2) Fix the columns
*https://en.wikipedia.org/wiki/Souls_(series)

In [1511]:
def shorten_list(takeIn, takeOut):
    temp = takeIn.lower().split() # split all the words into a list
    temp2 = [word for word in temp if word.lower() not in takeOut] # create a shorter list of words minus the take-outs
    return ' '.join(temp2) # return that shorter list as a string

In [1512]:
tag_remove_list = ['japanese', 'checkpicture', 'complete', 'haspicture', 'nomemo',
                   'researched', 'aaaeditthis', 'addaudio', 'addaudio2', 'addaudioNow',
                   'addmore','adjustformatting', 'hascomparison', 'hasmnemonic',
                   'customediting','wikidefinition', 'givewill','addaudionow','addprompt',
                   'checknuance','giveyaneury','hastextimage', 'marked', 'addpicture',
                   'addexampletranslation','basicnumeric', 'genkiplus', 'hasaudio',
                   'nativeaudio', 'adddefinition','addexamples', 'addjapaneseprompt',
                   'computervoice','haspoliteprefix','nongoo','customdefinition','hashint',
                   'abahipriorityfix','kaki','mcd','nobodyknows+','missingwordtype',
                   'image','duplicate', 'hasprompt', 'ninshiki','abachecknuance',
                   'hasflag','things', 'jim', 'hasunicode', 'editthis','aaahipriority',
                   'hassimpledef', 'givecodie', 'forjimmy', 'hasnativeaudio', 'givejimmy2',
                   'checkaudio', 'checkwriting', 'hasjlptlevel', 'makekaki', 'checknuance2',
                   'checkagain', 'newaudio', 'mail', 'checkexamples','elementaryschool',
                   'nvc', 'checkprompt', 'gavejimmy', 'addnativeaudio','checkreading',
                   'givecodieapril', 'activated', 'fixformatting','hasplacesuffix',
                   'hassuffix','addtranslation','addnewcardtype','addnuance','addtextimage',
                   'semicomplete', 'removeroboaudio','fixaudio','hasgramconj', 
                   'addkanji','changenotetype', 'famous', 'kuverb',
                   'givwill','karutapoems', 'map', 'hasvisualcomparison','picturekaki',
                   'jyugemu', '2018', 'type1', 'hasslang', 'apologies',
                   'month', 'definitionresearched','soundshift', 'basics1', 'tsuverb',
                   'facebook', 'uverb', 'checkfrequency', 'degree', 'hasdefinition',
                   'addtransliteration', 'dnd', 'introductions', 'adjustprompt',
                   'job', 'particle', 'services', 'mature', 'splitpictures', 
                   'egaki', 'type5k', 'intimate','extrainfo', 'irregular', 'unlisted',
                   'fromwiki', 'checkdifference','addpronunciationdiagram', 'reset',
                   'currentevents', 'doubletextimage', 'comparison', 'verbscompoundpast2',
                   'attention', 'addmemo', 'averb', 'radio','hasascii', 'fontadjusted',
                   'haspronunciation', 'borroweddefinition','alphabet', 'graphics',
                   'chiebukuro', 'duolingo', 'ateji', 'fact','type5s', 'fixpicture',
                   'politebydefault', 'objects','sensitive', 'groupword', 'addmnemonic',
                   'hasmore', 'quote', 'checkformatting','overlap', 'kotobankdef',
                   'hasrudeness', 'changedeck', 'specialformatting','yoga',
                   'hasjapaneseprompt', 'hasprefix','questionword', 'business', 
                   'postoffice', 'firstten', 'money', 'robotvoice2', 'ichidan', 'godan',
                   'weather','count', 'nodefinition', 'muverb', 'addcomparisonchart', 
                   'ruverb', 'phone', 'conjugated','haddiv','vulgar','fromkaruta',
                   'karutamanual', 'teform', '2019', 'onecharacter'
                  ]

### 2.2.9. Remove unneeded tags (meta-data) from notes

In [1513]:
# survey a few notes to see example tag data
df_notes.head(3)

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,Sound,Sound2,Sound3,Examples,ExamplesAudio,AtoQ,AtoQaudio,AtoQkana,AtoQtranslation,QandApicture,answerPicture,Meaning1,SimilarWords,RelatedWords,Breakdown1,Comparison,Usage,Prompt1,Prompt2,KakuMCD,IuMCD,ExtraMemo,Yomi2,Meaning2,Breakdown2,Picture1,Picture2,Picture3,Picture4,HinshiMarker,Hint,Term2,ArabicNumeral,CounterKanji,Mnemonic,SameSoundWords,Yomi3,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2
0,1331799797110,1557777642,Marked abaCheckNuance checkNuance complete ka...,臨機応変,りんきおうへん,adapting oneself to the requirements of the mo...,,,,"Noun, No-adjective",,,,,,,,,,,,［名・形動］その時その場に応じて、適切な手段をとること。また、そのさま。「―な（の）処置」「...,,,,,"<!--StartFragment--><div class=""prog_derivativ...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1331799797112,1557777642,complete kanji noMemo researched wwwjdic,隙間,すきま,<div>crevice; crack; gap; opening</div>,,,,"<div>Common word, Noun</div>",,,,,,,,,,,,<div>１ 物と物との間の、わずかにあいている所。「―だらけの板塀」</div><div>...,すき,空隙(くうげき),,,"<!--StartFragment--><div class=""prog_block"" st...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,1331799797113,1557777642,Marked abaCheckNuance checkNuance complete ka...,苦汁,にがり,bittern; concentrated solution of salts (esp. ...,,,,Noun,,,,,,,,,,,,海水から食塩を析出させたあとの残液。苦みがあり、主成分は塩化マグネシウム。豆腐の凝固剤などに...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [1514]:
# likely useful tags: katakana, Waseigo, Food, Phrases, casual, restaurant, travel, commonWord, noun, suruVerb

df_notes_001_less_tags = df_notes.copy() #originally "df_notes_less_tags"
df_notes_001_less_tags['tags'] = df_notes_001_less_tags['tags'].apply(lambda x: shorten_list(str(x), tag_remove_list))

print_before_after(df_notes['tags'].iloc[0], df_notes_001_less_tags['tags'].iloc[0],"Tags for " + df_notes['Term'].iloc[0])

assertEquals("researched" in df_notes['tags'].iloc[0].split(), True, "Contains Tag 'researched'")
assertEquals("researched" in df_notes_001_less_tags['tags'].iloc[0].split(), False, "Contains Tag 'researched'")

---------------------------------------------------------------------------
Tags for 臨機応変
---------------------------------------------------------------------------
Before:  Marked abaCheckNuance checkNuance complete kanji noMemo researched wwwjdic yojijukugo 
---------------------------------------------------------------------------
After: kanji wwwjdic yojijukugo
---------------------------------------------------------------------------


'OK'

### 2.2.10. Rename useful tags (meta-data) that were poorly named (still on notes)

In [1515]:
# replace list (formerly named 'tag_replace_list')
tag_rename_dict = {
    'aalowfrequency':'rare checked', 'aatechnical':'technical checked', 'aaanonkaiwa':'nonconvo checked',
    'wwwjdic':'fromdict', 'expression':'phrase', 'numberonly':'number',
    'grammarpoint':'grammar', 'jisho':'fromdict', 'pointingword':'directions',
    'geometry':'math technical', 'genki':'textbook', 'jpn202':'college',
    'jpn201':'college', 'jpn101':'college', 'jpn102':'college', 'kentei':'fromexam',
    'proficiencytest':'fromexam', 'bodypart':'body', '5kyuu':'fromexam',
    'linguisticreference':'technical', 'conversation':'convo',
    'fromconvo':'convo', 'culturepoint':'culture', 'checkednuance':'checked',
    'checkedpictures':'checked', 'checkednuance':'checked', 'medical':'technical',
    'anatomy':'body', 'places':'place', 'animals':'animal',
    'newspaperterm':'fromnewspaper', 'checkedreading':'checked',
    'abbreviation':'abbr','firstsemester':'semester1', 'verbs':'verb',
    'convook':'checked convo','inuse':'checked',
    'nuancechecked':'checked','insects':'animal insect','sightseeing':'travel',
    'accessories':'clothing', 'grammarsuffix':'suffix', 'oceanlife':'animal ocean',
    'science':'technical', 'written':'nonconvo', 'notrare':'checked',
    'aajoke':'silly', 'intonationcompare':'hassimilar', 'ij':'textbook',
    'goodcard':'inspect','aahilevel':'challenging inspect', 'ijvocab':'textbook',
    'cliothing':'clothing','unused':'nonconvo rare checked',
    'aaunused':'nonconvo rare checked', 'samesound':'hassame','animals':'animal',
    'dictionary':'fromdict','usuallywritteninkana':'kana',
    'abVeryRare':'rare checked', 'yojijukugo':'rare idiom', 'abcasual':'casual checked convo',
    'literaryform':'nonconvo', 'onomatopoeiclike':'onomatopoeic','kenjo':'humble',
    'colors':'color', 'forest':'nature','flower':'plant nature', 'aaok':'checked',
    'questions': 'question', 'adverbs':'adverb','book2':'textbook',
    'book1':'textbook','proficiencytest':'fromtest','animalscomplete':'animal',
    'sonkei':'respectful','eating':'food','fruit':'food','neverused':'nonconvo rare',
    'domainspecific':'technical','seaons':'season','seasons':'season',
    'prefecture':'place','plantpart':'plant', "hakataben":"dialect", "fish":"animal fish",
    "transitive":"transitive verb", "intransitive":"intransitive verb",
    "aaunecessary":"nonconvo checked", "vegetables":"vegetable food plant",
    "counters":"counter", "senmonyougo":"technical", "countries":"country place",
    "date":"datesandtime", "rarelyused":"rare", "aaakaiwa":"convo checked", "cool":"inspect",
    "investigate":"inspect","challenging":"inspect","names":"name",'qanda':'question',
    'hasquestion':'question', "感情のもとにあったニーズ":"phrase rare","phrases":'phrase',
    'iadjective':'iadj adj', 'naajective':'naadj adj', 'adverbs':'adv', 'adverb':'adv'
} # 'onecharacter':'len1',

#todo: investigate:
#editformatting,  datesandtime, linguistics, reference, adult, adjustpicture, checkpronunciation, addhint, challenging, inspect

In [1516]:
def replace_list(takeIn, replaceDict):
    temp = takeIn.lower().split()
    temp2 = []
    for word in temp:
        if word in replaceDict:
            temp2.append(replaceDict.get(word)) # if the word exists in the dictionary, replace it
        else:
            temp2.append(word) # if the word doesnt't exist in the dictionary, leave it alone
    return ' '.join(temp2) # return that shorter list as a string

# inspect further:
# multiwriting, multimeaning, multipicture, multiterm, multireading, mergeterms, checkpronunciation, customterm,
# goodcard, personalized, silly, addjlptlevel, checkpronunciation, mergeterms, customterm, transportation vs travel

# categorize: iadjective, naajective, verb, counter, commonword, suruverb, pronoun, question, phrases, kuverb, godan, ichidan, intransitive, transitive, noun, adverbialnoun

In [1517]:
df_notes_002_better_tags = df_notes_001_less_tags.copy() # originally "df_notes_better_tags"
df_notes_002_better_tags['tags'] = df_notes_002_better_tags['tags'].apply(lambda x: replace_list(str(x), tag_rename_dict))

print_before_after(df_notes_001_less_tags['tags'].iloc[0], df_notes_002_better_tags['tags'].iloc[0], "Tags for " + df_notes_002_better_tags['Term'].iloc[0])

assertEquals("wwwjdic" in df_notes_001_less_tags['tags'].iloc[0].split(), True, "Contains Tag 'wwwjdic'")
assertEquals("wwwjdic" in df_notes_002_better_tags['tags'].iloc[0].split(), False, "Contains Tag 'wwwjdic'")
assertEquals("fromdict" in df_notes_001_less_tags['tags'].iloc[0].split(), False, "Contains Tag 'fromdict'")
assertEquals("fromdict" in df_notes_002_better_tags['tags'].iloc[0].split(), True, "Contains Tag 'fromdict'")

---------------------------------------------------------------------------
Tags for 臨機応変
---------------------------------------------------------------------------
Before: kanji wwwjdic yojijukugo
---------------------------------------------------------------------------
After: kanji fromdict rare idiom
---------------------------------------------------------------------------


'OK'

### 2.2.11. Inspect current tag strings, notice duplicate occurances

In [1518]:
df_notes_002_better_tags['tags'].value_counts()[:5]

kanji                      1474
kanji fromtest textbook     547
fromdict kanji              430
kanji fromdict              302
textbook kanji textbook     210
Name: tags, dtype: int64

### 2.2.12. Add "notags" tag to notes w/o any meta-tag data

In [1519]:
df_notes_002_better_tags['tags'] = df_notes_002_better_tags['tags'].apply(lambda x: "hasnotags" if x == '' else x)

In [1520]:
df_notes_002_better_tags['tags'].value_counts()[:5]

kanji                      1474
kanji fromtest textbook     547
fromdict kanji              430
kanji fromdict              302
textbook kanji textbook     210
Name: tags, dtype: int64

We can attempt to inspect which tags are most common, in which combinations, and which words would be ideal
for further additional metadata. However, **our tags are still lumped together** at this point. Also, there is
reason to believe that **some tags are showing up multiple times in the same tag string**. In order to properly count tag frequency, the duplicates must be confirmed absent (ie. found & removed). Then, the occurance (word frequency) of each tag may then be summed up for the tags column.

### 2.2.13. Inspect a note suspected for tag duplication

In [1521]:
# confirm that a particular note has tag duplicates
# crimison note id: 1369286386384
note_id_1 = 1369286386384
assertEquals(get_rows_by_value_in_col(df_notes_002_better_tags, note_id_1,'nid').tags.values[0].count('fromexam'),
             2,"Two occurances of the tag 'fromexam' exist")

#todo: count occurances of 'fromexam' instead

'OK'

In [1522]:
# example of item with tag duplication
sel2 = get_rows_by_value_in_col(df_notes_002_better_tags, note_id_1,'nid')
sel2

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,Sound,Sound2,Sound3,Examples,ExamplesAudio,AtoQ,AtoQaudio,AtoQkana,AtoQtranslation,QandApicture,answerPicture,Meaning1,SimilarWords,RelatedWords,Breakdown1,Comparison,Usage,Prompt1,Prompt2,KakuMCD,IuMCD,ExtraMemo,Yomi2,Meaning2,Breakdown2,Picture1,Picture2,Picture3,Picture4,HinshiMarker,Hint,Term2,ArabicNumeral,CounterKanji,Mnemonic,SameSoundWords,Yomi3,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2
3845,1369286386384,1557777649,fromexam color kanji fromexam,紅,くれない,<div>deep red; crimson</div><div><br /></div><...,,,,"<div>Common word, Noun</div><div>Common word, ...",,,,,,,,,,,,<div>《「くれ（呉）のあい（藍）」の音変化》</div><div>１ 鮮明な赤色。特に、...,,,,,deep red; crimson<div><br></div><div>(all read...,（くれない・べに）,,,,,べに,,,"<img src=""104739.jpg"" />",,,,,,,,,,,,,,,,,,,,


### 2.2.14. Remove duplicate tags (convert tag strings > lists > sets > strings)

In [1523]:
# Converts a tag string to a list to a set back to a string (this removes the duplicates)
def remove_dupes(t):
    temp = list(set(t.lower().split()))
    return ' '.join(temp) # return as string

In [1524]:
df_notes_003_tags_no_dups = df_notes_002_better_tags.copy()
df_notes_003_tags_no_dups['tags'] = df_notes_003_tags_no_dups['tags'].apply(lambda x: remove_dupes(str(x)))

In [1525]:
# determines if an individual tag substring exists in a larger tags list string
def tag_exists(tags, tag):
    return 1 if tag in tags.split() else 0

In [1526]:
print(get_rows_by_value_in_col(df_notes_003_tags_no_dups, note_id_1,'nid').tags.values[0])
assertEquals(tag_exists(get_rows_by_value_in_col(df_notes_003_tags_no_dups, note_id_1,'nid').tags.values[0],"color"), 1, "tag 'color' remains")
assertEquals(tag_exists(get_rows_by_value_in_col(df_notes_003_tags_no_dups, note_id_1,'nid').tags.values[0],"fromexam"), 1, "tag 'fromexam' remains")
assertEquals(tag_exists(get_rows_by_value_in_col(df_notes_003_tags_no_dups, note_id_1,'nid').tags.values[0],"kanji"), 1, "tag 'kanji' remains")

color fromexam kanji


'OK'

It appears we have most, if not all, of the data we need to start. The format of the dates though is not yet human readable. Let's fix that.

### 2.2.15. Convert (& preserve) note ID to note creation date

In [1527]:
#dueNum = 782 # this represents days from collection creation date
#crt = 1357635600 # this represents the collection creation date #todo: query dynamically from database
#print("mid 'model id': " + time.ctime(int("1768161991"))) # 1 day = 86400 seconds

df_notes_004_with_date = df_notes_003_tags_no_dups.copy()
df_notes_004_with_date['NoteCreated']= pd.to_datetime(df_notes_004_with_date['nid'],unit='ms')
#df_notes_004_with_date['NoteCreated'] = df_notes_004_with_date['NoteCreated'].dt.date
df_notes_004_with_date.head()

print_before_after(df_notes_003_tags_no_dups['nid'].iloc[0], df_notes_004_with_date['NoteCreated'].iloc[0],"Term " + df_notes_004_with_date['Term'].iloc[0])

#assertEquals(df_notes_004_with_date['nid'].iloc[0], 1331799797110, "Note ID is in Epoch Units")
#assertEquals(str(df_notes_004_with_date['NoteCreated'].iloc[0]), "2012-03-15", "Note ID is in datetime date format year-month-day")

---------------------------------------------------------------------------
Term 臨機応変
---------------------------------------------------------------------------
Before: 1331799797110
---------------------------------------------------------------------------
After: 2012-03-15 08:23:17.110000
---------------------------------------------------------------------------


In [1528]:
df_notes_004_with_date.dtypes

nid                         int64
mod                         int64
tags                       object
Term                       object
Yomi1                      object
Translation                object
Translation2               object
Translation3               object
AlternateForms             object
PartOfSpeech               object
Sound                      object
Sound2                     object
Sound3                     object
Examples                   object
ExamplesAudio              object
AtoQ                       object
AtoQaudio                  object
AtoQkana                   object
AtoQtranslation            object
QandApicture               object
answerPicture              object
Meaning1                   object
SimilarWords               object
RelatedWords               object
Breakdown1                 object
Comparison                 object
Usage                      object
Prompt1                    object
Prompt2                    object
KakuMCD       

### 2.2.15. Generate Note Last Modified Date from "Mod" ID

In [1529]:
df_notes_005_last_modified = df_notes_004_with_date.copy()
df_notes_005_last_modified['LastModified'] = pd.to_datetime(df_notes_005_last_modified['mod'],unit='s')
#df_notes_005_last_modified['LastModified'] = df_notes_005_last_modified['LastModified'].dt.date

#assertEquals(str(df_notes_005_last_modified['LastModified'].iloc[0]), "2017-11-23", "Note last modified is in datetime date format year-month-day")

### 2.2.16. Remove rare words, phrases, expressions, questions & sentences from notes

In [1530]:
df_notes_006_only_vocab_no_rare = df_notes_005_last_modified.copy()
print(df_notes_006_only_vocab_no_rare.shape)
df_notes_006_only_vocab_no_rare.head(3)

(8375, 58)


,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,Sound,Sound2,Sound3,Examples,ExamplesAudio,AtoQ,AtoQaudio,AtoQkana,AtoQtranslation,QandApicture,answerPicture,Meaning1,SimilarWords,RelatedWords,Breakdown1,Comparison,Usage,Prompt1,Prompt2,KakuMCD,IuMCD,ExtraMemo,Yomi2,Meaning2,Breakdown2,Picture1,Picture2,Picture3,Picture4,HinshiMarker,Hint,Term2,ArabicNumeral,CounterKanji,Mnemonic,SameSoundWords,Yomi3,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2,NoteCreated,LastModified
0,1331799797110,1557777642,fromdict rare idiom kanji,臨機応変,りんきおうへん,adapting oneself to the requirements of the mo...,,,,"Noun, No-adjective",,,,,,,,,,,,［名・形動］その時その場に応じて、適切な手段をとること。また、そのさま。「―な（の）処置」「...,,,,,"<!--StartFragment--><div class=""prog_derivativ...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2012-03-15 08:23:17.110,2019-05-13 20:00:42
1,1331799797112,1557777642,fromdict kanji,隙間,すきま,<div>crevice; crack; gap; opening</div>,,,,"<div>Common word, Noun</div>",,,,,,,,,,,,<div>１ 物と物との間の、わずかにあいている所。「―だらけの板塀」</div><div>...,すき,空隙(くうげき),,,"<!--StartFragment--><div class=""prog_block"" st...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2012-03-15 08:23:17.112,2019-05-13 20:00:42
2,1331799797113,1557777642,fromdict kanji,苦汁,にがり,bittern; concentrated solution of salts (esp. ...,,,,Noun,,,,,,,,,,,,海水から食塩を析出させたあとの残液。苦みがあり、主成分は塩化マグネシウム。豆腐の凝固剤などに...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2012-03-15 08:23:17.113,2019-05-13 20:00:42


In [1531]:
sel4 = df_notes_006_only_vocab_no_rare[df_notes_006_only_vocab_no_rare['tags'].str.contains("rare")]
# https://stackoverflow.com/questions/37313691/how-to-remove-a-pandas-dataframe-from-another-dataframe
# remove rare words only first
df_notes_006_only_vocab_no_rare = pd.concat([df_notes_006_only_vocab_no_rare, sel4]).drop_duplicates(keep=False)

print(df_notes_006_only_vocab_no_rare.shape)
df_notes_006_only_vocab_no_rare.head(3)

# todo: assert that no rare words remain in 'df_notes_006_only_vocab_no_rare' by using 'contain("rare")'
# for selection, assert that selection has a row size of 0

(8263, 58)


,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,Sound,Sound2,Sound3,Examples,ExamplesAudio,AtoQ,AtoQaudio,AtoQkana,AtoQtranslation,QandApicture,answerPicture,Meaning1,SimilarWords,RelatedWords,Breakdown1,Comparison,Usage,Prompt1,Prompt2,KakuMCD,IuMCD,ExtraMemo,Yomi2,Meaning2,Breakdown2,Picture1,Picture2,Picture3,Picture4,HinshiMarker,Hint,Term2,ArabicNumeral,CounterKanji,Mnemonic,SameSoundWords,Yomi3,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2,NoteCreated,LastModified
1,1331799797112,1557777642,fromdict kanji,隙間,すきま,<div>crevice; crack; gap; opening</div>,,,,"<div>Common word, Noun</div>",,,,,,,,,,,,<div>１ 物と物との間の、わずかにあいている所。「―だらけの板塀」</div><div>...,すき,空隙(くうげき),,,"<!--StartFragment--><div class=""prog_block"" st...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2012-03-15 08:23:17.112,2019-05-13 20:00:42
2,1331799797113,1557777642,fromdict kanji,苦汁,にがり,bittern; concentrated solution of salts (esp. ...,,,,Noun,,,,,,,,,,,,海水から食塩を析出させたあとの残液。苦みがあり、主成分は塩化マグネシウム。豆腐の凝固剤などに...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2012-03-15 08:23:17.113,2019-05-13 20:00:42
3,1331799797114,1557777642,fromdict kanji,移籍,いせき,<div>changing household registry; transfer (e....,,,,"<div>Common word, Noun, Suru verb</div>",,,,,,,,,,,,<div>［名］(スル)</div><div>１ 婚姻・養子縁組などで、ある人が、ある戸籍か...,,,,,"<!--StartFragment--><div class=""prog_block"" st...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2012-03-15 08:23:17.114,2019-05-13 20:00:42


### 2.2.17. Remove phrases, sentences, questions, & grammar (hasTilde) cards all at once

In [1532]:
sel5 = df_notes_006_only_vocab_no_rare[df_notes_006_only_vocab_no_rare['tags'].str.contains("phrase")]
sel6 = df_notes_006_only_vocab_no_rare[df_notes_006_only_vocab_no_rare['tags'].str.contains("sentence")]
sel7 = df_notes_006_only_vocab_no_rare[df_notes_006_only_vocab_no_rare['tags'].str.contains("question")]
sel01 = df_notes_006_only_vocab_no_rare[df_notes_006_only_vocab_no_rare['tags'].str.contains("hasTilde")]
df_notes_006_only_vocab_no_rare = pd.concat([df_notes_006_only_vocab_no_rare, sel5, sel6, sel7, sel01]).drop_duplicates(keep=False)

print(df_notes_006_only_vocab_no_rare.shape)
df_notes_006_only_vocab_no_rare.head(3)

# todo: assert that no rare, phrase, sentence or question cards remain in the collection of notes

(8026, 58)


,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,Sound,Sound2,Sound3,Examples,ExamplesAudio,AtoQ,AtoQaudio,AtoQkana,AtoQtranslation,QandApicture,answerPicture,Meaning1,SimilarWords,RelatedWords,Breakdown1,Comparison,Usage,Prompt1,Prompt2,KakuMCD,IuMCD,ExtraMemo,Yomi2,Meaning2,Breakdown2,Picture1,Picture2,Picture3,Picture4,HinshiMarker,Hint,Term2,ArabicNumeral,CounterKanji,Mnemonic,SameSoundWords,Yomi3,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2,NoteCreated,LastModified
1,1331799797112,1557777642,fromdict kanji,隙間,すきま,<div>crevice; crack; gap; opening</div>,,,,"<div>Common word, Noun</div>",,,,,,,,,,,,<div>１ 物と物との間の、わずかにあいている所。「―だらけの板塀」</div><div>...,すき,空隙(くうげき),,,"<!--StartFragment--><div class=""prog_block"" st...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2012-03-15 08:23:17.112,2019-05-13 20:00:42
2,1331799797113,1557777642,fromdict kanji,苦汁,にがり,bittern; concentrated solution of salts (esp. ...,,,,Noun,,,,,,,,,,,,海水から食塩を析出させたあとの残液。苦みがあり、主成分は塩化マグネシウム。豆腐の凝固剤などに...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2012-03-15 08:23:17.113,2019-05-13 20:00:42
3,1331799797114,1557777642,fromdict kanji,移籍,いせき,<div>changing household registry; transfer (e....,,,,"<div>Common word, Noun, Suru verb</div>",,,,,,,,,,,,<div>［名］(スル)</div><div>１ 婚姻・養子縁組などで、ある人が、ある戸籍か...,,,,,"<!--StartFragment--><div class=""prog_block"" st...",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,2012-03-15 08:23:17.114,2019-05-13 20:00:42


The note model has a bunch of columns (fields) with no values in them. These can be taken out for data analysis.

In [1533]:
# let's look a a small slice of data, to infer what we may
# we can take a broad overview look at the dataset to more quickly isolate candidates for removal
s = get_rows_by_value_in_col(df_notes_006_only_vocab_no_rare, '発明','Term')
s.head()

,nid,mod,tags,Term,Yomi1,Translation,Translation2,Translation3,AlternateForms,PartOfSpeech,Sound,Sound2,Sound3,Examples,ExamplesAudio,AtoQ,AtoQaudio,AtoQkana,AtoQtranslation,QandApicture,answerPicture,Meaning1,SimilarWords,RelatedWords,Breakdown1,Comparison,Usage,Prompt1,Prompt2,KakuMCD,IuMCD,ExtraMemo,Yomi2,Meaning2,Breakdown2,Picture1,Picture2,Picture3,Picture4,HinshiMarker,Hint,Term2,ArabicNumeral,CounterKanji,Mnemonic,SameSoundWords,Yomi3,gChap,gBook,semester,gNumber,Transliteration,SoloLookCards,TagOverflow,blank1,blank2,NoteCreated,LastModified
2696,1354094556789,1557777647,textbook suruverb commonword fromtest n3 noun ...,発明,はつめい,"<span style=""""><div>invention</div></span>",,,,"Common word, Noun, Suru verb",[sound:pronunciation_ja_発明.mp3][sound:pronunci...,,,飛行機を発明した人は誰ですか？<div>ひこうき を はつめい した ひと は だれ です ...,,,,,,,,<div>［名］(スル)</div><div>１ 今までなかったものを新たに考え出すこと。特...,,,,,"<div><div><!--StartFragment--><div class=""prog...",<div><br /></div>,<div><br /></div>,<div><div>発(めい)する</div><div>彼には発(めい)の才がある</div...,<div>（？）する</div><div>彼には（？）の才がある</div><div>必要は...,,,,,"<img src=""flywheelbicycle-11.jpg"" />","<img src=""chronicle-of-flight1.jpg"" />","<img src=""x_5f4187ab.jpg"" />","<img src=""mercedes1_b2.jpg"" />",,,,,,,,,,,,,,,,,,2012-11-28 09:22:36.789,2019-05-13 20:00:47


### 2.2.18. Determine which columns (fields) are unused & can be safely removed

In [1534]:
def is_blank (s):
    return not (s and s.strip())

In [1535]:
col_names = df_notes_006_only_vocab_no_rare.columns.values
# see that this cell for this row is indeed blank
#print(is_blank(df_notes_006_only_vocab_no_rare['Translation2'].iloc[0]))

row_cnt = df_notes_006_only_vocab_no_rare.shape[0] # number of rows in df_notes_006_only_vocab_no_rare

# https://stackoverflow.com/questions/49677060/pandas-count-empty-strings-in-a-column
empty_strings = pd.DataFrame(df_notes_006_only_vocab_no_rare.values == '',columns=col_names) # find all empty strings in a DataFrame
temp_dict = (empty_strings.sum()).to_dict()  # save the location of all empty strings as a DataFrame of booleans
removal_candidates = []
for key in temp_dict.items():
    if key[1] == row_cnt:
        removal_candidates.append(key[0])
print("Removal candidates:", removal_candidates)

Removal candidates: ['Sound3', 'AtoQ', 'AtoQaudio', 'AtoQkana', 'AtoQtranslation', 'QandApicture', 'answerPicture', 'blank1', 'blank2']


### 2.2.19. Drop empty columns from notes data frame

In [1536]:
df_notes_007_less_cols = df_notes_006_only_vocab_no_rare.copy()

df_notes_007_less_cols = df_notes_007_less_cols.drop(removal_candidates,axis=1)

print_before_after(df_notes_006_only_vocab_no_rare.shape, df_notes_007_less_cols.shape)
print_before_after(df_notes_006_only_vocab_no_rare.columns.values, df_notes_007_less_cols.columns.values)

---------------------------------------------------------------------------
Before: (8026, 58)
---------------------------------------------------------------------------
After: (8026, 49)
---------------------------------------------------------------------------
---------------------------------------------------------------------------
Before: ['nid' 'mod' 'tags' 'Term' 'Yomi1' 'Translation' 'Translation2'
 'Translation3' 'AlternateForms' 'PartOfSpeech' 'Sound' 'Sound2' 'Sound3'
 'Examples' 'ExamplesAudio' 'AtoQ' 'AtoQaudio' 'AtoQkana'
 'AtoQtranslation' 'QandApicture' 'answerPicture' 'Meaning1'
 'SimilarWords' 'RelatedWords' 'Breakdown1' 'Comparison' 'Usage' 'Prompt1'
 'Prompt2' 'KakuMCD' 'IuMCD' 'ExtraMemo' 'Yomi2' 'Meaning2' 'Breakdown2'
 'Picture1' 'Picture2' 'Picture3' 'Picture4' 'HinshiMarker' 'Hint' 'Term2'
 'ArabicNumeral' 'CounterKanji' 'Mnemonic' 'SameSoundWords' 'Yomi3'
 'gChap' 'gBook' 'semester' 'gNumber' 'Transliteration' 'SoloLookCards'
 'TagOverflow' 'blank1' 'blank2

### 2.2.20. Create binary exists/not columns based on presence of a given tag in notes data frame

In [1537]:
def add_column_by_tag(df, tag):
    df[tag] = df['tags'].apply(lambda x: tag_exists(str(x), tag))

In [1538]:
df_notes_008_binary_tags = df_notes_007_less_cols.copy()
inspect_list = ["commonword",
                "clothing", "animal", "body", "food", "place",
                "textbook", "college", "fromdict", "fromexam",
                "n1", "n2", "n3", "n4", "n5",
                'katakana','hiragana','kanji',
                'adv', 'adj', 'noun', 'verb',
                'nonconvo', 'convo','hasnotags'
               ] # todo: for next time, inspect "rare" tag
for item in inspect_list:
    add_column_by_tag(df_notes_008_binary_tags, item)

In [1539]:
df_notes_008_binary_tags.dtypes.value_counts()

object            45
int64             27
datetime64[ns]     2
dtype: int64

### 2.2.21. Create boolean columns in notes data frame for predictive models

In [1540]:
# https://stackoverflow.com/questions/17383094/how-can-i-map-true-false-to-1-0-in-a-pandas-dataframe
#df_notes_008_binary_tags['hasPOS'] = df_notes_008_binary_tags['PartOfSpeech']!="" #todo: expand upon this, by tagify
df_notes_008_binary_tags['hasVisual'] = df_notes_008_binary_tags['Picture1']!=""
#df_notes_008_binary_tags['hasReading'] = df_notes_008_binary_tags['Yomi1']!="" # todo: replace via 'kanji' tag
df_notes_008_binary_tags['hasAudio'] = df_notes_008_binary_tags['Sound']!=""
df_notes_008_binary_tags['hasMultiMeaning'] = df_notes_008_binary_tags['Translation2' and 'Translation3' and 'Meaning2']!=""
df_notes_008_binary_tags['hasMultiReading'] = df_notes_008_binary_tags['Yomi2']!="" # todo: inspect & incorporate venn diagram: https://commons.wikimedia.org/wiki/File:Homograph_homophone_venn_diagram.png
df_notes_008_binary_tags['hasSimilar'] = df_notes_008_binary_tags['SimilarWords']!=""
df_notes_008_binary_tags['hasHomophone'] = df_notes_008_binary_tags['SameSoundWords']!="" # write function, detect homophones
df_notes_008_binary_tags['hasAltForm'] = df_notes_008_binary_tags['Term2' and 'AlternateForms']!= ""
df_notes_008_binary_tags['hasRichExamples'] = df_notes_008_binary_tags['Examples' and 'ExamplesAudio']!=""

In [1541]:
# Laura calls this process "Data Enriching"
# todo: confirm that intify_list is to be different/same than inspect_list
intify_list = ['hasVisual','hasAudio','hasMultiMeaning','hasMultiReading',
               'hasSimilar','hasHomophone','hasAltForm','hasnotags','hasRichExamples'] # 'hasReading',

### 2.2.22. Drop non-numerical columns from notes data frame

In [1542]:
df_notes_009_less_cols = df_notes_008_binary_tags.copy()
df_notes_009_less_cols = df_notes_009_less_cols.drop(['Examples','ExamplesAudio',
                            'Meaning1','RelatedWords','Breakdown1','Comparison',
                           'Usage','Prompt1','Prompt2','KakuMCD','IuMCD','ExtraMemo',
                           'Breakdown2','Picture2','Picture3','Picture4','Mnemonic',
                            'Yomi3','gChap','gBook','semester','gNumber','ArabicNumeral',
                            'CounterKanji','SoloLookCards','HinshiMarker','Hint',
                            'mod','Transliteration','Picture1','Sound','Sound2',
                            'TagOverflow','Translation2', 'Meaning2','Yomi2','Term2',
                            'SameSoundWords','SimilarWords','AlternateForms',
                            'Translation3','Translation','PartOfSpeech'],axis=1) #'hasPOS',
# todo: explore 'mod' (last modified date) as freshness metric

In [1543]:
# casts columns of type object to types (such as int) as directed, use with caution
def cast_to_typ(df, col, typ):
    df[col] = df[col].astype(typ)

### 2.2.23. Enforce proper numerical boolean type encoding in notes data frame

In [1544]:
df_notes_009_less_cols.dtypes.value_counts()

int64             26
bool               8
object             3
datetime64[ns]     2
dtype: int64

In [1545]:
for item in intify_list:
    cast_to_typ(df_notes_009_less_cols,item, int)

In [1546]:
df_notes_009_less_cols.dtypes.value_counts()

int64             34
object             3
datetime64[ns]     2
dtype: int64

In [1547]:
df_notes_009_less_cols.head(35)[30:]

#selection2 = df_binary.loc[df_binary['hasMultiMeaning']==1]
#selection2.head()

,nid,tags,Term,Yomi1,NoteCreated,LastModified,commonword,clothing,animal,body,food,place,textbook,college,fromdict,fromexam,n1,n2,n3,n4,n5,katakana,hiragana,kanji,adv,adj,noun,verb,nonconvo,convo,hasnotags,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,hasRichExamples
36,1342506824728,fromdict fromtest textbook kanji,方法,ほうほう,2012-07-17 06:33:44.728,2019-05-13 20:00:42,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
37,1342506824729,fromdict kanji,行為,こうい,2012-07-17 06:33:44.729,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
38,1342506824730,fromtest textbook kanji,行動,こうどう,2012-07-17 06:33:44.730,2019-05-13 20:00:42,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
39,1342506824731,hassimilar hassame fromdict kanji,事態,じたい,2012-07-17 06:33:44.731,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0
40,1342506824732,suffix fromtest textbook kanji,形,かたち,2012-07-17 06:33:44.732,2019-05-13 20:00:42,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


### 2.2.24. Count syllables & character length for each term in notes data frame

In [1548]:
df_notes_010_with_len = df_notes_009_less_cols.copy()

df_notes_010_with_len['TermLen'] = df_notes_010_with_len['Term'].str.len()
df_notes_010_with_len['Syllables'] = df_notes_010_with_len['Yomi1'].str.len()
df_notes_010_with_len.loc[df_notes_010_with_len['Syllables'] == 0, 'Syllables'] = df_notes_010_with_len['TermLen']

bins = [0,1,2,4,8,128]
labels = ["[1]","[2]","[3:4]","[5:8]","[9: ]"]
# https://stackoverflow.com/questions/45273731/binning-column-with-python-pandas
df_notes_010_with_len['TermLenGroup'] = pd.cut(df_notes_010_with_len['TermLen'], bins=bins, labels=labels)

# example: df.loc[df['Grades'] <= 77, 'Grades'] = 100
# https://stackoverflow.com/questions/42815768/pandas-adding-column-with-the-length-of-other-column-as-value
df_notes_010_with_len.tail(20)[:5]

,nid,tags,Term,Yomi1,NoteCreated,LastModified,commonword,clothing,animal,body,food,place,textbook,college,fromdict,fromexam,n1,n2,n3,n4,n5,katakana,hiragana,kanji,adv,adj,noun,verb,nonconvo,convo,hasnotags,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,hasRichExamples,TermLen,Syllables,TermLenGroup
8345,1485703825555,multiintonation commonword katakana gairaigo n...,ズボン,,2017-01-29 15:30:25.555,2019-05-13 19:58:21,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,3,3,[3:4]
8346,1485705343576,commonword katakana n3 noun gairaigo clothing,ベルト,,2017-01-29 15:55:43.576,2019-05-13 19:58:21,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,3,3,[3:4]
8347,1485705402623,commonword katakana gairaigo noun clothing,ブラジャー,,2017-01-29 15:56:42.623,2019-05-13 19:58:21,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,5,5,[5:8]
8348,1489373157595,kanji,細切り,ほそぎり,2017-03-13 02:45:57.595,2019-05-13 20:00:56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,4,[3:4]
8349,1489756408272,kanji,離陸,りりく,2017-03-17 13:13:28.272,2019-05-13 20:00:56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,[2]


### 2.2.25. Inspect the longest syllable entries in notes data frame

In [1549]:
df_many_syl = df_notes_010_with_len.copy()
many_syl = df_many_syl['Syllables'] > 16
df_many_syl.loc[many_syl] #todo: check nid of 1391477462767

,nid,tags,Term,Yomi1,NoteCreated,LastModified,commonword,clothing,animal,body,food,place,textbook,college,fromdict,fromexam,n1,n2,n3,n4,n5,katakana,hiragana,kanji,adv,adj,noun,verb,nonconvo,convo,hasnotags,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,hasRichExamples,TermLen,Syllables,TermLenGroup
242,1346057958628,fromdict history inspect fromnewspaper culture...,東京電力福島・第１原発事故,とうきょうでんりょくふくしま・だいいちげんぱつじこ,2012-08-27 08:59:18.628,2019-05-13 20:00:43,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,13,25,[9: ]
308,1346215143756,datesandtime numeric,1837～1901年,せんはっぴゃくさんじゅうななねんからせんきゅうひゃくいちねん,2012-08-29 04:39:03.756,2019-05-13 20:58:19,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,30,[9: ]
421,1346216471844,fromdict counter numeric datesandtime kanji,千九百八十九年,せんきゅうひゃくはちじゅうきゅうねん,2012-08-29 05:01:11.844,2019-05-13 20:00:43,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,18,[5:8]
5542,1387411183585,datesandtime numeric kanji,千九百八十七年,せんきゅうひゃくはちじゅうななねん,2013-12-18 23:59:43.585,2019-05-13 20:00:52,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,17,[5:8]


In [1550]:
df_notes_010_with_len.columns.values

array(['nid', 'tags', 'Term', 'Yomi1', 'NoteCreated', 'LastModified',
       'commonword', 'clothing', 'animal', 'body', 'food', 'place',
       'textbook', 'college', 'fromdict', 'fromexam', 'n1', 'n2', 'n3',
       'n4', 'n5', 'katakana', 'hiragana', 'kanji', 'adv', 'adj', 'noun',
       'verb', 'nonconvo', 'convo', 'hasnotags', 'hasVisual', 'hasAudio',
       'hasMultiMeaning', 'hasMultiReading', 'hasSimilar', 'hasHomophone',
       'hasAltForm', 'hasRichExamples', 'TermLen', 'Syllables',
       'TermLenGroup'], dtype=object)

In [1551]:
# labels terms by their jlpt level.
# bear in mind that some terms have multiple jlpt levels.
# this function merely assigns the lowest associated jlpt level with a term. 
def label_jlpt_lvl (row):
    if row['n5'] == 1 :
        return 5
    elif row['n4'] == 1:
        return 4
    elif row['n3'] == 1:
        return 3
    elif row['n2'] == 1:
        return 2
    elif row['n1'] == 1:
        return 1
    else:
        return None

### 2.2.26. Assign JLPT number to words with JLPT "N" levels in notes data frame

In [1552]:
df_notes_011_jptl_lvl = df_notes_010_with_len.copy()
df_notes_011_jptl_lvl['jlpt_lvl_d'] = df_notes_011_jptl_lvl.apply (lambda row: label_jlpt_lvl(row), axis=1)

In [1553]:
df_notes_011_jptl_lvl.columns.values

array(['nid', 'tags', 'Term', 'Yomi1', 'NoteCreated', 'LastModified',
       'commonword', 'clothing', 'animal', 'body', 'food', 'place',
       'textbook', 'college', 'fromdict', 'fromexam', 'n1', 'n2', 'n3',
       'n4', 'n5', 'katakana', 'hiragana', 'kanji', 'adv', 'adj', 'noun',
       'verb', 'nonconvo', 'convo', 'hasnotags', 'hasVisual', 'hasAudio',
       'hasMultiMeaning', 'hasMultiReading', 'hasSimilar', 'hasHomophone',
       'hasAltForm', 'hasRichExamples', 'TermLen', 'Syllables',
       'TermLenGroup', 'jlpt_lvl_d'], dtype=object)

In [1554]:
df_notes_011_jptl_lvl['jlpt_lvl_d'].value_counts()

3.0    197
5.0    155
4.0    104
1.0     81
2.0     77
Name: jlpt_lvl_d, dtype: int64

In [1555]:
# 2.2.26.2 Assign script (char) type to each note
df_notes_012_script_type = df_notes_011_jptl_lvl.copy()

char_list = ['katakana','hiragana'] # ,'hasReading'

In [1556]:
def label_script (row):
    if row['katakana'] == 1 :
        return 'katakana'
    elif row['hiragana'] == 1:
        return 'hiragana'
    elif row['kanji'] == 1:
        return 'kanji'
    else:
        return 'otherScript' # todo: add 'romanji' type

In [1557]:
df_notes_012_script_type['script'] = df_notes_012_script_type.apply (lambda row: label_script(row), axis=1)

In [1558]:
df_notes_012_script_type.head()

,nid,tags,Term,Yomi1,NoteCreated,LastModified,commonword,clothing,animal,body,food,place,textbook,college,fromdict,fromexam,n1,n2,n3,n4,n5,katakana,hiragana,kanji,adv,adj,noun,verb,nonconvo,convo,hasnotags,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,hasRichExamples,TermLen,Syllables,TermLenGroup,jlpt_lvl_d,script
1,1331799797112,fromdict kanji,隙間,すきま,2012-03-15 08:23:17.112,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,3,[2],NaN,kanji
2,1331799797113,fromdict kanji,苦汁,にがり,2012-03-15 08:23:17.113,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,[2],NaN,kanji
3,1331799797114,fromdict kanji,移籍,いせき,2012-03-15 08:23:17.114,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,[2],NaN,kanji
5,1331799797117,fromdict verb kanji,吊るす,つるす,2012-03-15 08:23:17.117,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,3,3,[3:4],NaN,kanji
6,1331799797118,convo checked fromdict kanji,和やか,なごやか,2012-03-15 08:23:17.118,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,4,[3:4],NaN,kanji


### 2.2.27. Create df_notes_012_final_section_2 data frame for progress saving

In [1559]:
#df_combo_005_notes_galore
df_notes_013_mid_section_2 = df_notes_012_script_type.copy()
print(df_notes_013_mid_section_2.shape)
df_notes_013_mid_section_2.head()

(8026, 44)


,nid,tags,Term,Yomi1,NoteCreated,LastModified,commonword,clothing,animal,body,food,place,textbook,college,fromdict,fromexam,n1,n2,n3,n4,n5,katakana,hiragana,kanji,adv,adj,noun,verb,nonconvo,convo,hasnotags,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,hasRichExamples,TermLen,Syllables,TermLenGroup,jlpt_lvl_d,script
1,1331799797112,fromdict kanji,隙間,すきま,2012-03-15 08:23:17.112,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,3,[2],NaN,kanji
2,1331799797113,fromdict kanji,苦汁,にがり,2012-03-15 08:23:17.113,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,[2],NaN,kanji
3,1331799797114,fromdict kanji,移籍,いせき,2012-03-15 08:23:17.114,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,[2],NaN,kanji
5,1331799797117,fromdict verb kanji,吊るす,つるす,2012-03-15 08:23:17.117,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,3,3,[3:4],NaN,kanji
6,1331799797118,convo checked fromdict kanji,和やか,なごやか,2012-03-15 08:23:17.118,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,4,[3:4],NaN,kanji


### 2.2.28. ~~Export df_notes_midway~~

In [1560]:
#df_notes_013_mid_section_2.to_csv('datasets/df_notes_013_mid_section_2.csv')

***
- [Previous section: Notes](#notes)
- [Next section: Review Log](#revlog)
***
# <a name="combo"></a> Combo of Notes & Cards

### 2.3.1. Merge card & note data frames to conduct cross analysis

In [1561]:
# now that we have note id's for all the words, we can
# join together these separate dataframes
df_combo = pd.merge(df_notes_013_mid_section_2, df_cards_009_mid_section_2, on='nid')
print(df_combo.shape)

# remove from df_combo any fields that belong strictly only to cards, such as c_ivl_q, c_factor_q
df_combo = df_combo.drop(['c_ivl_q','c_factor_q'], axis=1)
df_combo.head()

(7957, 60)


,nid,tags,Term,Yomi1,NoteCreated,LastModified,commonword,clothing,animal,body,food,place,textbook,college,fromdict,fromexam,n1,n2,n3,n4,n5,katakana,hiragana,kanji,adv,adj,noun,verb,nonconvo,convo,hasnotags,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,hasRichExamples,TermLen,Syllables,TermLenGroup,jlpt_lvl_d,script,cid,ivl,factor,reps,lapses,CardCreated,DueDate,CardType_listen,CardType_look,CardType_read,CardType_recall,cardtype,efficiency,durability
0,1331799797112,fromdict kanji,隙間,すきま,2012-03-15 08:23:17.112,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,3,[2],NaN,kanji,1331799797112,149,2080,8,1,2012-03-15 08:23:17.112,2015-03-03 09:00:00,0,0,1,0,read,0.125000,18.625000
1,1331799797114,fromdict kanji,移籍,いせき,2012-03-15 08:23:17.114,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,[2],NaN,kanji,1331799797114,99,1980,7,0,2012-03-15 08:23:17.114,2015-02-04 09:00:00,0,0,1,0,read,0.000000,14.142857
2,1331799797117,fromdict verb kanji,吊るす,つるす,2012-03-15 08:23:17.117,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,3,3,[3:4],NaN,kanji,1331799797117,143,2130,6,1,2012-03-15 08:23:17.117,2015-03-17 09:00:00,0,0,1,0,read,0.166667,23.833333
3,1331799797118,convo checked fromdict kanji,和やか,なごやか,2012-03-15 08:23:17.118,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,4,[3:4],NaN,kanji,1331799797118,74,1880,15,3,2012-03-15 08:23:17.118,2015-02-06 09:00:00,0,0,1,0,read,0.200000,4.933333
4,1331799797121,fromdict kanji,営業日,えいぎょうび,2012-03-15 08:23:17.121,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,6,[3:4],NaN,kanji,1331799797121,132,2130,6,1,2012-03-15 08:23:17.121,2015-03-03 09:00:00,0,0,1,0,read,0.166667,22.000000


Let's further refine the dataframe entries to represent which notes have (1) visual data, (2) audio data, and (3) a L1 ("first language", English in this case) translation. We can represent these with binary values (0 for doesn't exist, 1 for exists).

### 2.3.2. Group notes by ID to determine card type overlap, simple totals per note

In [1562]:
# https://stackoverflow.com/questions/23919563/merge-rows-of-a-dataframe-in-pandas-based-on-a-column
# https://stackoverflow.com/questions/13851535/how-to-delete-rows-from-a-pandas-dataframe-based-on-a-conditional-expression
df_combo_001_grouped_notes = df_combo.copy()
###### remove from this temporary grouped notes dataframe any card specific items, such as card interval or card factor (ease factor)
df_combo_001_grouped_notes = df_combo_001_grouped_notes.drop(
    ['cid','hasAltForm','TermLen','Syllables','jlpt_lvl_d','katakana','hiragana',
     'noun','verb','convo','commonword','clothing','animal','body','food','place','textbook',
     'college','fromdict','fromexam','n1','n2','n3','n4','n5','hasVisual','hasAudio',
     'hasMultiMeaning','hasMultiReading','hasSimilar','hasHomophone','hasRichExamples',
     'hasnotags','efficiency','durability'],axis=1) # ,'hasReading','len1'

# create data frame for totals per term: reps, lapses, and all vectors (card types) 
df_combo_002_note_totals = df_combo_001_grouped_notes.copy()
df_combo_002_note_totals = df_combo_002_note_totals.groupby(['nid']).sum()

# drop numerical fields that don't logically add up (interval, ease factor, and the quartile/quintile buckets for interval and ease factor)
df_combo_002_note_totals = df_combo_002_note_totals.drop(['ivl','factor'],axis=1) # 'c_ivl_q','c_factor_q'
df_combo_002_note_totals.tail(20)[-5:]

,kanji,adv,adj,nonconvo,reps,lapses,CardType_listen,CardType_look,CardType_read,CardType_recall
nid,,,,,,,,,,
1517489889767,1,0,0,0,2,0,0,1,0,0
1523892839900,4,0,0,0,33,4,1,1,1,1
1549184119039,2,0,0,0,8,0,0,0,1,1
1550402953788,2,0,0,0,13,2,0,0,1,1
1550403040864,2,0,0,0,7,0,0,0,1,1


In [1563]:
df_combo_002_note_totals = df_combo_002_note_totals.rename(
    columns={
        'reps':'total_reps', 'lapses':'total_lapses', 'CardType_listen':'hasListenCard',
        'CardType_recall':'hasTranslateCard', 'CardType_read':'hasReadCard',
        'CardType_look':'hasPictureCard'
    }
)

In [1564]:
df_combo_002_note_totals.tail(20)[-5:]

,kanji,adv,adj,nonconvo,total_reps,total_lapses,hasListenCard,hasPictureCard,hasReadCard,hasTranslateCard
nid,,,,,,,,,,
1517489889767,1,0,0,0,2,0,0,1,0,0
1523892839900,4,0,0,0,33,4,1,1,1,1
1549184119039,2,0,0,0,8,0,0,0,1,1
1550402953788,2,0,0,0,13,2,0,0,1,1
1550403040864,2,0,0,0,7,0,0,0,1,1


### 2.3.3. Group notes by ID to find simple average means per note

In [1565]:
df_combo_001_grouped_notes.tail(20)[-5:]

,nid,tags,Term,Yomi1,NoteCreated,LastModified,kanji,adv,adj,nonconvo,TermLenGroup,script,ivl,factor,reps,lapses,CardCreated,DueDate,CardType_listen,CardType_look,CardType_read,CardType_recall,cardtype
7952,1549184119039,kanji,閏年,うるうどし,2019-02-03 08:55:19.039,2019-05-13 20:00:56,1,0,0,0,[2],kanji,87,2410,3,0,2019-02-06 01:06:25.907,2019-07-24 09:00:00,0,0,0,1,recall
7953,1550402953788,fromdict convo suruverb commonword noun kanji,段取り,だんどり,2019-02-17 11:29:13.788,2019-05-13 20:00:56,1,0,0,0,[3:4],kanji,91,2410,4,0,2019-02-17 11:30:09.251,2019-08-04 09:00:00,0,0,1,0,read
7954,1550402953788,fromdict convo suruverb commonword noun kanji,段取り,だんどり,2019-02-17 11:29:13.788,2019-05-13 20:00:56,1,0,0,0,[3:4],kanji,9,2010,9,2,2019-02-17 11:30:09.269,2019-05-14 09:00:00,0,0,0,1,recall
7955,1550403040864,kanji,触角,しょっかく,2019-02-17 11:30:40.864,2019-05-13 20:00:56,1,0,0,0,[2],kanji,82,2410,3,0,2019-02-17 11:31:24.990,2019-07-19 09:00:00,0,0,1,0,read
7956,1550403040864,kanji,触角,しょっかく,2019-02-17 11:30:40.864,2019-05-13 20:00:56,1,0,0,0,[2],kanji,93,2410,4,0,2019-02-17 11:31:25.003,2019-08-06 09:00:00,0,0,0,1,recall


In [1566]:
df_combo_003_note_means = df_combo_001_grouped_notes.copy()
df_combo_003_note_means = df_combo_003_note_means.groupby(['nid']).mean()
df_combo_003_note_means.tail(20)[-5:]

,kanji,adv,adj,nonconvo,ivl,factor,reps,lapses,CardType_listen,CardType_look,CardType_read,CardType_recall
nid,,,,,,,,,,,,
1517489889767,1.0,0.0,0.0,0.0,1.0,2410.0,2.00,0.0,0.00,1.00,0.00,0.00
1523892839900,1.0,0.0,0.0,0.0,336.0,2220.0,8.25,1.0,0.25,0.25,0.25,0.25
1549184119039,1.0,0.0,0.0,0.0,97.5,2410.0,4.00,0.0,0.00,0.00,0.50,0.50
1550402953788,1.0,0.0,0.0,0.0,50.0,2210.0,6.50,1.0,0.00,0.00,0.50,0.50
1550403040864,1.0,0.0,0.0,0.0,87.5,2410.0,3.50,0.0,0.00,0.00,0.50,0.50


In [1567]:
df_combo_003_note_means = df_combo_003_note_means.drop(['CardType_listen','CardType_recall',
    'CardType_read', 'CardType_look'],axis=1)
df_combo_003_note_means = df_combo_003_note_means.rename(
    columns={'ivl':'mean_ivl','factor':'mean_factor','reps':'mean_reps','lapses':'mean_lapses'})
df_combo_003_note_means.tail(20)[-5:]

,kanji,adv,adj,nonconvo,mean_ivl,mean_factor,mean_reps,mean_lapses
nid,,,,,,,,
1517489889767,1.0,0.0,0.0,0.0,1.0,2410.0,2.00,0.0
1523892839900,1.0,0.0,0.0,0.0,336.0,2220.0,8.25,1.0
1549184119039,1.0,0.0,0.0,0.0,97.5,2410.0,4.00,0.0
1550402953788,1.0,0.0,0.0,0.0,50.0,2210.0,6.50,1.0
1550403040864,1.0,0.0,0.0,0.0,87.5,2410.0,3.50,0.0


### 2.3.4. Combine note totals, note means & general notes

In [1568]:
df_temp = df_combo_002_note_totals.copy()
df_temp = df_temp.drop(['kanji','adv','adj','nonconvo'],axis=1)

df_temp_2 = df_combo_003_note_means.copy()
df_temp_2 = df_temp_2.drop(['kanji','adv','adj','nonconvo'],axis=1) # 'ivl_q','factor_q'

df_combo_004_notes_galore = pd.merge(df_temp_2, df_temp,on='nid')
#print(df_combo_004_notes_galore.columns.values)

df_combo_005_notes_galore = pd.merge(df_combo,df_combo_004_notes_galore,on='nid')
# https://stackoverflow.com/questions/47022070/display-all-dataframe-columns-in-a-jupyter-python-notebook
pd.options.display.max_columns = None

#print(df_combo_005_notes_galore.columns.values)

# drop card specific columns, these are no longer valid
df_combo_005_notes_galore = df_combo_005_notes_galore.drop(['cid','ivl','factor','reps','lapses','CardCreated','DueDate','CardType_listen','CardType_look','CardType_read','CardType_recall'],axis=1)
df_combo_005_notes_galore.head(10)

,nid,tags,Term,Yomi1,NoteCreated,LastModified,commonword,clothing,animal,body,food,place,textbook,college,fromdict,fromexam,n1,n2,n3,n4,n5,katakana,hiragana,kanji,adv,adj,noun,verb,nonconvo,convo,hasnotags,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,hasRichExamples,TermLen,Syllables,TermLenGroup,jlpt_lvl_d,script,cardtype,efficiency,durability,mean_ivl,mean_factor,mean_reps,mean_lapses,total_reps,total_lapses,hasListenCard,hasPictureCard,hasReadCard,hasTranslateCard
0,1331799797112,fromdict kanji,隙間,すきま,2012-03-15 08:23:17.112,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,3,[2],NaN,kanji,read,0.125000,18.625000,149.0,2080.0,8.0,1.0,8,1,0,0,1,0
1,1331799797114,fromdict kanji,移籍,いせき,2012-03-15 08:23:17.114,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,[2],NaN,kanji,read,0.000000,14.142857,99.0,1980.0,7.0,0.0,7,0,0,0,1,0
2,1331799797117,fromdict verb kanji,吊るす,つるす,2012-03-15 08:23:17.117,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,3,3,[3:4],NaN,kanji,read,0.166667,23.833333,143.0,2130.0,6.0,1.0,6,1,0,0,1,0
3,1331799797118,convo checked fromdict kanji,和やか,なごやか,2012-03-15 08:23:17.118,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,4,[3:4],NaN,kanji,read,0.200000,4.933333,74.0,1880.0,15.0,3.0,15,3,0,0,1,0
4,1331799797121,fromdict kanji,営業日,えいぎょうび,2012-03-15 08:23:17.121,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,6,[3:4],NaN,kanji,read,0.166667,22.000000,132.0,2130.0,6.0,1.0,6,1,0,0,1,0
5,1331799797122,fromdict hassame kanji,在庫,ざいこ,2012-03-15 08:23:17.122,2019-05-13 21:42:11,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,3,[2],NaN,kanji,read,0.000000,44.800000,224.0,2130.0,5.0,0.0,5,0,0,0,1,0
6,1331799797126,fromdict kanji,有能,ゆうのう,2012-03-15 08:23:17.126,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,4,[2],NaN,kanji,read,0.000000,27.555556,248.0,2130.0,9.0,0.0,9,0,0,0,1,0
7,1331799797127,fromdict waseigo katakana kanji,公衆トイレ,こうしゅうトイレ,2012-03-15 08:23:17.127,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,5,8,[5:8],NaN,katakana,read,0.000000,25.444444,407.5,2245.0,9.0,0.0,18,0,0,1,1,0
8,1331799797127,fromdict waseigo katakana kanji,公衆トイレ,こうしゅうトイレ,2012-03-15 08:23:17.127,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,5,8,[5:8],NaN,katakana,look,0.000000,65.111111,407.5,2245.0,9.0,0.0,18,0,0,1,1,0
9,1331799797128,fromdict kanji,送り賃,おくりちん,2012-03-15 08:23:17.128,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,5,[3:4],NaN,kanji,read,0.000000,22.250000,178.0,2120.0,8.0,0.0,8,0,0,0,1,0


### 2.3.5. Inspect combo dtypes

In [1569]:
# strategies to fix dtypes: https://stackoverflow.com/questions/28910851/python-pandas-changing-some-column-types-to-categories
print(df_combo_005_notes_galore.dtypes.value_counts())
df_combo_005_notes_galore.dtypes

int64             38
float64            7
object             5
uint8              4
datetime64[ns]     2
category           1
dtype: int64


nid                          int64
tags                        object
Term                        object
Yomi1                       object
NoteCreated         datetime64[ns]
LastModified        datetime64[ns]
commonword                   int64
clothing                     int64
animal                       int64
body                         int64
food                         int64
place                        int64
textbook                     int64
college                      int64
fromdict                     int64
fromexam                     int64
n1                           int64
n2                           int64
n3                           int64
n4                           int64
n5                           int64
katakana                     int64
hiragana                     int64
kanji                        int64
adv                          int64
adj                          int64
noun                         int64
verb                         int64
nonconvo            

### 2.3.6. Fix combo dtypes

In [1570]:
convert_bool_list = ['hasRichExamples']

for item in convert_bool_list:
    cast_to_typ(df_combo_005_notes_galore,item, int)

convert_category_list = ['jlpt_lvl_d']

for col in convert_category_list:
    df_combo_005_notes_galore[col] = df_combo_005_notes_galore[col].astype('category')

convert_dates_list = ['NoteCreated','LastModified']

# https://stackoverflow.com/questions/28910851/python-pandas-changing-some-column-types-to-categories
for col in convert_dates_list:
    df_combo_005_notes_galore[col] = df_combo_005_notes_galore[col].astype('datetime64')
    
# todo: use the following three lists as a rough guide to start prepping dataframes
# for export & analysis in section 3!!! ^_^
binary_list = ['commonword','clothing','animal','body','food','place','textbook','college',
    'fromdict','fromexam','n1','n2','n3','n4','n5','katakana','hiragana','noun','verb',
    'convo','hasnotags','hasVisual','hasAudio','hasMultiMeaning','hasMultiReading','hasSimilar',
    'hasHomophone','hasAltForm','hasRichExamples','hasListenCard','hasPictureCard','hasReadCard',
    'hasTranslateCard'] # 'len1'
continuous_list = ['TermLen','Syllables','mean_ivl','mean_factor','mean_reps','mean_lapses',
                   'total_reps','total_lapses']
discrete_non_binary_list = ['NoteCreated','LastModified','TermLenGroup','jlpt_lvl_d']
    
df_combo_005_notes_galore.dtypes # todo: convert 'cardtype' column dtype to 'category'

nid                          int64
tags                        object
Term                        object
Yomi1                       object
NoteCreated         datetime64[ns]
LastModified        datetime64[ns]
commonword                   int64
clothing                     int64
animal                       int64
body                         int64
food                         int64
place                        int64
textbook                     int64
college                      int64
fromdict                     int64
fromexam                     int64
n1                           int64
n2                           int64
n3                           int64
n4                           int64
n5                           int64
katakana                     int64
hiragana                     int64
kanji                        int64
adv                          int64
adj                          int64
noun                         int64
verb                         int64
nonconvo            

### 2.3.7. Create df_combo_006_final_section_2 data frame for export

In [1571]:
df_combo_006_final_section_2 = df_combo_005_notes_galore.copy()
print(df_combo_006_final_section_2.shape)
df_combo_006_final_section_2.tail(10)[:5]

(7957, 57)


,nid,tags,Term,Yomi1,NoteCreated,LastModified,commonword,clothing,animal,body,food,place,textbook,college,fromdict,fromexam,n1,n2,n3,n4,n5,katakana,hiragana,kanji,adv,adj,noun,verb,nonconvo,convo,hasnotags,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,hasRichExamples,TermLen,Syllables,TermLenGroup,jlpt_lvl_d,script,cardtype,efficiency,durability,mean_ivl,mean_factor,mean_reps,mean_lapses,total_reps,total_lapses,hasListenCard,hasPictureCard,hasReadCard,hasTranslateCard
7947,1523892839900,noun commonword n5 kanji,万年筆,まんねんひつ,2018-04-16 15:33:59.900,2019-05-13 20:00:56,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,3,6,[3:4],5.0,kanji,read,0.142857,60.000000,336.0,2220.0,8.25,1.0,33,4,1,1,1,1
7948,1523892839900,noun commonword n5 kanji,万年筆,まんねんひつ,2018-04-16 15:33:59.900,2019-05-13 20:00:56,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,3,6,[3:4],5.0,kanji,listen,0.000000,103.000000,336.0,2220.0,8.25,1.0,33,4,1,1,1,1
7949,1523892839900,noun commonword n5 kanji,万年筆,まんねんひつ,2018-04-16 15:33:59.900,2019-05-13 20:00:56,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,3,6,[3:4],5.0,kanji,recall,0.142857,57.857143,336.0,2220.0,8.25,1.0,33,4,1,1,1,1
7950,1523892839900,noun commonword n5 kanji,万年筆,まんねんひつ,2018-04-16 15:33:59.900,2019-05-13 20:00:56,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,3,6,[3:4],5.0,kanji,look,0.142857,0.285714,336.0,2220.0,8.25,1.0,33,4,1,1,1,1
7951,1549184119039,kanji,閏年,うるうどし,2019-02-03 08:55:19.039,2019-05-13 20:00:56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,5,[2],NaN,kanji,read,0.000000,21.600000,97.5,2410.0,4.00,0.0,8,0,0,0,1,1


### 2.3.8. ~~Export df_combo_006_final_section_2~~

In [1572]:
#df_combo_006_final_section_2.to_csv('datasets/df_combo_006_final_section_2.csv')

In [1573]:
# todo: recombine the mid section notes dataframe w/ df_combo_006_final_section2
# the goal here is to avoid unnecessary row data duplication & keep all columns

In [1574]:
#df_temp_3 = df_notes_013_mid_section_2.copy()
#df_temp_3 = df_temp_3.drop(['tags','Term','Yomi1','NoteCreated','LastModified','commonword',
#    'clothing', 'animal', 'body', 'food', 'place','textbook', 'college', 'fromdict', 'fromexam',
#    'len1', 'n1', 'n2','n3', 'n4', 'n5', 'katakana', 'hiragana', 'noun', 'verb', 'convo',
#    'hasnotags', 'hasVisual', 'hasReading','hasAudio','hasMultiMeaning', 'hasMultiReading',
#    'hasSimilar', 'hasHomophone','hasAltForm', 'hasRichExamples', 'TermLen', 'Syllables',
#    'TermLenGroup','jlpt_lvl_d'],axis=1)

In [1575]:
#df_temp_3.head()

In [1576]:
# todo: remove extraneous cell & variable renaming
df_notes_013_final_section_2 = df_combo_006_final_section_2.copy()

In [1577]:
df_notes_014_final_section_2 = df_notes_013_final_section_2.copy()
df_notes_014_final_section_2 = df_notes_014_final_section_2.drop(['efficiency','durability','cardtype'],axis=1) #'ivl_q','factor_q'
df_notes_014_final_section_2 = df_notes_014_final_section_2.drop_duplicates(['nid'], keep='first')

In [1578]:
df_notes_014_final_section_2.tail(20)[-5:] ####

,nid,tags,Term,Yomi1,NoteCreated,LastModified,commonword,clothing,animal,body,food,place,textbook,college,fromdict,fromexam,n1,n2,n3,n4,n5,katakana,hiragana,kanji,adv,adj,noun,verb,nonconvo,convo,hasnotags,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,hasRichExamples,TermLen,Syllables,TermLenGroup,jlpt_lvl_d,script,mean_ivl,mean_factor,mean_reps,mean_lapses,total_reps,total_lapses,hasListenCard,hasPictureCard,hasReadCard,hasTranslateCard
7946,1517489889767,kanji,絵文字,えもじ,2018-02-01 12:58:09.767,2019-05-13 20:00:56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,3,3,[3:4],NaN,kanji,1.0,2410.0,2.00,0.0,2,0,0,1,0,0
7947,1523892839900,noun commonword n5 kanji,万年筆,まんねんひつ,2018-04-16 15:33:59.900,2019-05-13 20:00:56,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,3,6,[3:4],5.0,kanji,336.0,2220.0,8.25,1.0,33,4,1,1,1,1
7951,1549184119039,kanji,閏年,うるうどし,2019-02-03 08:55:19.039,2019-05-13 20:00:56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,5,[2],NaN,kanji,97.5,2410.0,4.00,0.0,8,0,0,0,1,1
7953,1550402953788,fromdict convo suruverb commonword noun kanji,段取り,だんどり,2019-02-17 11:29:13.788,2019-05-13 20:00:56,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,3,4,[3:4],NaN,kanji,50.0,2210.0,6.50,1.0,13,2,0,0,1,1
7955,1550403040864,kanji,触角,しょっかく,2019-02-17 11:30:40.864,2019-05-13 20:00:56,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,5,[2],NaN,kanji,87.5,2410.0,3.50,0.0,7,0,0,0,1,1


**reps** = work done to remember a card  
**interval** = memory length as output of memorization work done  
**ease/factor** = indicator of effort to retreive & store memory  
**lapses** = result of memory deficit, a common side-effect & indicator of inefficiency of memorization efforts  

**lapses/reps ratio** (efficiency ratio) => the closer to 0, the better ("low waste"). the higher this is, the worse : "high waste"  
**interval/reps ratio** (durability ratio) = the higher the better ("low effort" / "sticky"). the lower this is, the worse ("high effort", "slippery")  

In [1579]:
df_notes_15_final_section_2 = df_notes_014_final_section_2.copy()
df_notes_15_final_section_2['mean_card_efficiency'] = df_notes_15_final_section_2['mean_lapses'] / df_notes_15_final_section_2['mean_reps']
df_notes_15_final_section_2['mean_note_efficiency'] = df_notes_15_final_section_2['total_lapses'] / df_notes_15_final_section_2['total_reps']
df_notes_15_final_section_2['mean_card_durability'] = df_notes_15_final_section_2['mean_ivl'] / df_notes_15_final_section_2['mean_reps']
df_notes_15_final_section_2['mean_note_durability'] = df_notes_15_final_section_2['mean_ivl'] / df_notes_15_final_section_2['total_reps']


In [1580]:
print(df_notes_15_final_section_2.columns.values)
df_notes_15_final_section_2.head()

['nid' 'tags' 'Term' 'Yomi1' 'NoteCreated' 'LastModified' 'commonword'
 'clothing' 'animal' 'body' 'food' 'place' 'textbook' 'college' 'fromdict'
 'fromexam' 'n1' 'n2' 'n3' 'n4' 'n5' 'katakana' 'hiragana' 'kanji' 'adv'
 'adj' 'noun' 'verb' 'nonconvo' 'convo' 'hasnotags' 'hasVisual' 'hasAudio'
 'hasMultiMeaning' 'hasMultiReading' 'hasSimilar' 'hasHomophone'
 'hasAltForm' 'hasRichExamples' 'TermLen' 'Syllables' 'TermLenGroup'
 'jlpt_lvl_d' 'script' 'mean_ivl' 'mean_factor' 'mean_reps' 'mean_lapses'
 'total_reps' 'total_lapses' 'hasListenCard' 'hasPictureCard'
 'hasReadCard' 'hasTranslateCard' 'mean_card_efficiency'
 'mean_note_efficiency' 'mean_card_durability' 'mean_note_durability']


,nid,tags,Term,Yomi1,NoteCreated,LastModified,commonword,clothing,animal,body,food,place,textbook,college,fromdict,fromexam,n1,n2,n3,n4,n5,katakana,hiragana,kanji,adv,adj,noun,verb,nonconvo,convo,hasnotags,hasVisual,hasAudio,hasMultiMeaning,hasMultiReading,hasSimilar,hasHomophone,hasAltForm,hasRichExamples,TermLen,Syllables,TermLenGroup,jlpt_lvl_d,script,mean_ivl,mean_factor,mean_reps,mean_lapses,total_reps,total_lapses,hasListenCard,hasPictureCard,hasReadCard,hasTranslateCard,mean_card_efficiency,mean_note_efficiency,mean_card_durability,mean_note_durability
0,1331799797112,fromdict kanji,隙間,すきま,2012-03-15 08:23:17.112,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,2,3,[2],NaN,kanji,149.0,2080.0,8.0,1.0,8,1,0,0,1,0,0.125000,0.125000,18.625000,18.625000
1,1331799797114,fromdict kanji,移籍,いせき,2012-03-15 08:23:17.114,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,3,[2],NaN,kanji,99.0,1980.0,7.0,0.0,7,0,0,0,1,0,0.000000,0.000000,14.142857,14.142857
2,1331799797117,fromdict verb kanji,吊るす,つるす,2012-03-15 08:23:17.117,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,3,3,[3:4],NaN,kanji,143.0,2130.0,6.0,1.0,6,1,0,0,1,0,0.166667,0.166667,23.833333,23.833333
3,1331799797118,convo checked fromdict kanji,和やか,なごやか,2012-03-15 08:23:17.118,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,3,4,[3:4],NaN,kanji,74.0,1880.0,15.0,3.0,15,3,0,0,1,0,0.200000,0.200000,4.933333,4.933333
4,1331799797121,fromdict kanji,営業日,えいぎょうび,2012-03-15 08:23:17.121,2019-05-13 20:00:42,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,6,[3:4],NaN,kanji,132.0,2130.0,6.0,1.0,6,1,0,0,1,0,0.166667,0.166667,22.000000,22.000000


In [1581]:
# Export actual final notes

In [1582]:
df_notes_15_final_section_2.to_csv('datasets/df_notes_015_final_section_2.csv')

In [1583]:
# Compare the note export columns to the card export columns
note_cols = df_notes_15_final_section_2.columns.values ######
card_cols = df_cards_009_mid_section_2.columns.values

In [1584]:
# https://www.geeksforgeeks.org/python-intersection-two-lists/
def intersection(lst1, lst2):
    temp = set(lst2) 
    lst3 = [value for value in lst1 if value in temp] 
    return lst3

In [1585]:
def difference(list1, list2):
    s = set(list2)
    return [x for x in list1 if x not in s]

In [1586]:
# See what matches between CARD and NOTE dataframes

In [1587]:
print("both cards and notes have:")
print(intersection(note_cols, card_cols))

both cards and notes have:
['nid']


In [1588]:
# See what differs between CARD and NOTE dataframes

In [1589]:
diff = difference(note_cols, card_cols)
diff2 = difference(card_cols, note_cols)
print("only cards has:")
print(intersection(card_cols, diff2))
print("only notes has:")
print(intersection(note_cols, diff))

only cards has:
['cid', 'ivl', 'factor', 'reps', 'lapses', 'CardCreated', 'DueDate', 'c_ivl_q', 'c_factor_q', 'CardType_listen', 'CardType_look', 'CardType_read', 'CardType_recall', 'cardtype', 'efficiency', 'durability']
only notes has:
['tags', 'Term', 'Yomi1', 'NoteCreated', 'LastModified', 'commonword', 'clothing', 'animal', 'body', 'food', 'place', 'textbook', 'college', 'fromdict', 'fromexam', 'n1', 'n2', 'n3', 'n4', 'n5', 'katakana', 'hiragana', 'kanji', 'adv', 'adj', 'noun', 'verb', 'nonconvo', 'convo', 'hasnotags', 'hasVisual', 'hasAudio', 'hasMultiMeaning', 'hasMultiReading', 'hasSimilar', 'hasHomophone', 'hasAltForm', 'hasRichExamples', 'TermLen', 'Syllables', 'TermLenGroup', 'jlpt_lvl_d', 'script', 'mean_ivl', 'mean_factor', 'mean_reps', 'mean_lapses', 'total_reps', 'total_lapses', 'hasListenCard', 'hasPictureCard', 'hasReadCard', 'hasTranslateCard', 'mean_card_efficiency', 'mean_note_efficiency', 'mean_card_durability', 'mean_note_durability']


***
- [Previous section: Combo of Notes & Cards](#combo)
- [To to bottom](#bottom)
***
# <a name="revlog"></a> Review Log

### 2.4.1. Import in Review Log data

In [1590]:
df_revlog = pd.read_sql_query("SELECT * FROM revlog", cnx)

In [1591]:
print(df_revlog.shape)
df_revlog.head()

(114412, 9)


,id,cid,usn,ease,ivl,lastIvl,factor,time,type
0,1332393018515,1331799797110,0,1,0,0,2500,6673,0
1,1333279992123,1331799797110,0,4,8,0,2600,11656,0
2,1333280001016,1331799797112,0,4,8,0,2600,8887,0
3,1333280097922,1331799797113,0,1,0,0,2500,29162,0
4,1333280107916,1331799797114,0,4,8,0,2600,9987,0


In [1592]:
df_revlog_001_review_date = df_revlog.copy()
df_revlog_001_review_date = df_revlog_001_review_date.rename(columns={'id':'rid'})
df_revlog_001_review_date['ReviewDate']= pd.to_datetime(df_revlog_001_review_date['rid'],unit='ms')
#df_revlog_001_review_date['ReviewDate'] = df_revlog_001_review_date['ReviewDate'].dt.date
df_revlog_001_review_date.head()

#assertEquals(df_revlog_001_review_date['rid'].iloc[0], 1332393018515, "Note ID is in Epoch Units")
#assertEquals(str(df_revlog_001_review_date['ReviewDate'].iloc[0]), "2012-03-22", "Note ID is in datetime date format year-month-day")

,rid,cid,usn,ease,ivl,lastIvl,factor,time,type,ReviewDate
0,1332393018515,1331799797110,0,1,0,0,2500,6673,0,2012-03-22 05:10:18.515
1,1333279992123,1331799797110,0,4,8,0,2600,11656,0,2012-04-01 11:33:12.123
2,1333280001016,1331799797112,0,4,8,0,2600,8887,0,2012-04-01 11:33:21.016
3,1333280097922,1331799797113,0,1,0,0,2500,29162,0,2012-04-01 11:34:57.922
4,1333280107916,1331799797114,0,4,8,0,2600,9987,0,2012-04-01 11:35:07.916


In [1593]:
current_note_id = get_rows_by_value_in_col(df_cards_009_mid_section_2, df_revlog['cid'].iloc[0], 'cid')['nid'].iloc[0]

# get_rows_by_value_in_col(df_notes_003_tags_no_dups, note_id_1,'nid')
#print("Note ID: ", get_rows_by_value_in_col(df_notes_013_mid_section_2,current_note_id,'nid'))

#get_rows_by_value_in_col(df_notes_013_mid_section_2,1331799797110,'nid').shape

#print("Term: ", get_rows_by_value_in_col(df_notes_013_mid_section_2,current_note_id,'nid')['Term'].iloc[0])
#print("Translation: ", get_rows_by_value_in_col(df_notes_013_mid_section_2,current_note_id,'nid')['Translation'].iloc[0])

#inspect_card_by_id(df_cards_009_mid_section_2, df_revlog['cid'].iloc[0], 'id')
#get_rows_by_value_in_col(df_cards_009_mid_section_2, df_revlog['cid'].iloc[0],'cid')

In [1594]:
get_rows_by_value_in_col(df_revlog_001_review_date, df_revlog['cid'].iloc[0], 'cid')

,rid,cid,usn,ease,ivl,lastIvl,factor,time,type,ReviewDate
0,1332393018515,1331799797110,0,1,0,0,2500,6673,0,2012-03-22 05:10:18.515
1,1333279992123,1331799797110,0,4,8,0,2600,11656,0,2012-04-01 11:33:12.123
80362,1397571358201,1331799797110,4480,1,-60,-60,2500,4292,0,2014-04-15 14:15:58.201
80363,1397571360841,1331799797110,4480,2,-600,-60,2500,2636,0,2014-04-15 14:16:00.841
80364,1397571363081,1331799797110,4480,2,1,-600,2280,2238,0,2014-04-15 14:16:03.081
80371,1397622541113,1331799797110,4490,3,2,1,2280,4023,1,2014-04-16 04:29:01.113
83538,1400914850867,1331799797110,4958,2,12,2,2130,3323,1,2014-05-24 07:00:50.867
93044,1410177777778,1331799797110,6257,2,44,12,1980,2300,1,2014-09-08 12:02:57.778
98887,1414062295845,1331799797110,6748,2,51,44,1830,16176,1,2014-10-23 11:04:55.845
104047,1420285596480,1331799797110,7154,2,65,51,1680,11880,1,2015-01-03 11:46:36.480


In [1595]:
# todo: put all revlog data per card in a cell alongside each card in the cards data frame

### <a id="bottom"></a> Hi there! Want to go back [to the top](#top)